# Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from collections import Counter
from matplotlib_venn import venn2, venn3
from scipy import stats
from tableone import TableOne
import statsmodels.api as sm
from bs4 import BeautifulSoup
import re
import seaborn as sns
import scipy
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,roc_auc_score
warnings.simplefilter('ignore')
import time
import scipy.sparse
from joblib import dump, load
import statsmodels
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [3]:
UKBB_PATH = '../glaucoma_project/UKBB_Data/'
MAIN_UKBB_FILE = '../glaucoma_project/UKBB_Data/ukb49508.tab'
SECOND_UKBB_FILE = '../glaucoma_project/UKBB_Data_Basket3/ukb51745.tab'
MAIN_HTML_FILE = '../glaucoma_project/UKBB_Data/ukb49508.html'
SECOND_HTML_FILE = '../glaucoma_project/UKBB_Data_Basket3/ukb51745.html'
TOTAL_POP = 502419
TEST_SET_RANDOM_SPLIT = 42
CHUNK_SIZE = 10000

# Creating test set

In [39]:
train_idx, test_idx = train_test_split(range(TOTAL_POP),random_state=TEST_SET_RANDOM_SPLIT, test_size=0.1)

In [40]:
def get_dataframe_for_analysis(column_names, idx=train_idx):
    print('Preparing dataframe...')
    
    cols_in_main_file = pd.read_csv(MAIN_UKBB_FILE, sep='\t', nrows=1).columns
    cols_in_second_file = pd.read_csv(SECOND_UKBB_FILE, sep='\t', nrows=1).columns
    column_names_main = [col for col in column_names if col in cols_in_main_file]
    column_names_second = [col for col in column_names if (not col in cols_in_main_file) and (col in cols_in_second_file)]
    
    
    df_selected_cols = pd.DataFrame()
    for chunk in tqdm(pd.read_csv(MAIN_UKBB_FILE, sep='\t', chunksize=CHUNK_SIZE, usecols=column_names_main+['f.eid']), total=TOTAL_POP // CHUNK_SIZE):
        df_selected_cols = pd.concat([df_selected_cols, chunk])
        
    df_selected_cols_second = pd.DataFrame()
    for chunk in tqdm(pd.read_csv(SECOND_UKBB_FILE, sep='\t', chunksize=CHUNK_SIZE, usecols=column_names_second+['f.eid']), total=TOTAL_POP // CHUNK_SIZE):
        df_selected_cols_second = pd.concat([df_selected_cols_second, chunk])
    
    return pd.merge(df_selected_cols,df_selected_cols_second,how='left',on='f.eid').iloc[idx].reset_index(drop=True)

# Helpful code for field ids

In [41]:
file = open(MAIN_HTML_FILE, 'r', encoding='latin-1')
html_file = file.read()
soup = BeautifulSoup(html_file, "html.parser")
tables = soup.find_all('table')
rows_main = tables[1].find_all('tr')
file = open(SECOND_HTML_FILE, 'r', encoding='latin-1')
html_file = file.read()
soup = BeautifulSoup(html_file, "html.parser")
tables = soup.find_all('table')
rows_second = tables[1].find_all('tr')
rows = rows_main + rows_second

In [42]:
df_fields = pd.DataFrame()
fields = []
count = []
types = []
desc = []
current_type = None
current_desc = None
for row in rows:
    row_content = row.find_all('td')
    if len(row_content) > 2:
        fields.append(row_content[1].text.strip())
        count.append(row_content[2].text.strip())
        if len(row_content) > 4:
            current_type = row_content[3].text.strip()
            current_desc = row_content[4].text.strip()
        types.append(current_type)
        desc.append(current_desc)
        
df_fields['UDI'] = fields
df_fields['Count'] = [int(n) for n in count]
df_fields['Type'] = types
df_fields['Desc'] = desc
df_fields['Dating coding'] = df_fields['Desc'].apply(lambda x: re.sub('[^0-9]','', x[x.index('data-coding '):x.index(' comprises')]) if 'data-coding' in x else np.nan)
df_fields['Valued members'] = df_fields['Desc'].apply(lambda x: int(re.sub('[^0-9]','', x[x.index('comprises '):x.index('valued members')])) if 'data-coding' in x else np.nan)
df_fields['Field ID'] = df_fields['UDI'].apply(lambda x: 'f.'+re.sub('-','.',x))
df_fields['Desc Short'] = df_fields['Desc'].apply(lambda x: x[0:x.index('Uses data-coding')] if 'Uses data-coding' in x else x)

In [43]:
desc_unique = []
prev_desc = None
counter = 0
for desc in df_fields['Desc Short']:
    if prev_desc == desc:
        if counter == 0:
            desc_unique[-1] = desc+'_0'
        counter += 1
        desc_unique.append(desc+'_'+str(counter))
    else:
        counter = 0
        prev_desc = desc
        desc_unique.append(desc)
df_fields['Desc Unique'] = desc_unique

In [44]:
# Field ID to description 
id_to_desc = {id_:desc for id_,desc in zip(df_fields['Field ID'], df_fields['Desc Unique'])}
# Description to field ID
desc_to_id = {desc:id_ for desc,id_ in zip(df_fields['Desc Unique'], df_fields['Field ID'])}
# Field ID to field type
id_to_type = {id_:type_ for id_,type_ in zip(df_fields['Field ID'], df_fields['Type'])}

In [45]:
df_fields = df_fields.drop_duplicates('UDI').reset_index(drop=True)

In [46]:
coding_dicts = []
for df_coding in [pd.read_csv(UKBB_PATH+'UKBB_encodings/coding19.tsv', sep='\t'),
                  pd.read_csv(UKBB_PATH+'UKBB_encodings/coding3.tsv', sep='\t'),
                  pd.read_csv(UKBB_PATH+'UKBB_encodings/coding6.tsv', sep='\t'),
                  pd.read_csv(UKBB_PATH+'UKBB_encodings/coding4.tsv', sep='\t')]:
    coding_dicts.append({code:meaning for code,meaning in zip(df_coding['coding'], df_coding['meaning'])})

In [47]:
def feature_id_to_name(feature_id):
    if 'f.' in feature_id:
        if '_' in feature_id:
            return id_to_desc[feature_id[:feature_id.find('_')]] + feature_id[feature_id.find('_'):]
        else:
            return id_to_desc[feature_id]
    if 'ICD_10_' in feature_id:
        return coding_dicts[0][feature_id[7:]]
    if 'Self_Report_' in feature_id:
        return coding_dicts[2][int(feature_id[12:-2])]
    if 'Cancer_' in feature_id:
        return coding_dicts[1][int(feature_id[7:-2])]
    if 'Med_' in feature_id:
        return coding_dicts[3][int(feature_id[4:-2])]
    return 'FEATURE NOT FOUND'

In [48]:
glaucoma_diagnosis = pd.read_pickle(UKBB_PATH+'processed_data/Clinical Feature Selection/glaucoma_diagnosis_checkpoint.pkl')

# Importing fields

In [ ]:
df_chosen_fields = pd.read_excel(UKBB_PATH+'processed_data/chosen_fields.xlsx')

In [ ]:
self_reported_matrix = pd.read_pickle(UKBB_PATH+'processed_data/Clinical Feature Selection/self_reported_diagnosis_checkpoint_2.pkl')
icd_10_matrix = pd.read_pickle(UKBB_PATH+'processed_data/Clinical Feature Selection/icd10_checkpoint_2.pkl')
med_matrix = pd.read_pickle(UKBB_PATH+'processed_data/Clinical Feature Selection/meds_checkpoint_2.pkl')

In [ ]:
all_fields = list(df_chosen_fields['Pragmatic'].dropna().values) + list(df_chosen_fields['Specialised'].values)

In [ ]:
general_fields = [field.split('_')[0] for field in all_fields if field[0] == 'f']

In [ ]:
icd_10_fields = [field.split('_')[2] for field in all_fields if 'ICD_10_' in field]

In [18]:
self_reported_fields = [float(field.split('_')[2]) for field in all_fields if 'Self_Report_' in field]

In [19]:
med_fields = [float(field.split('_')[1]) for field in all_fields if 'Med_' in field]

In [20]:
assert len(med_fields) + len(self_reported_fields) + len(icd_10_fields) + len(general_fields) == len(all_fields)

In [21]:
general_matrix = get_dataframe_for_analysis(general_fields)

Preparing dataframe...


51it [05:01,  5.90s/it]                                                        
51it [00:38,  1.31it/s]                                                        


In [22]:
assert [f for f in general_fields if not f in general_matrix.columns] == []

In [23]:
self_reported_matrix = self_reported_matrix[self_reported_fields]
icd_10_matrix = icd_10_matrix[icd_10_fields]
med_matrix = med_matrix[med_fields]

# Cleaning columns

In [24]:
def clean_categorical_column_with_dummies(column):
    # Don't process things that aren't actually columns
    if type(column) == pd.core.frame.DataFrame:
        return pd.DataFrame()
    no_neg_list = []
    for item in column:
        try:
            if np.float64(item) < 0:
                no_neg_list.append(np.nan)
            else:
                no_neg_list.append(item)
        except:
            no_neg_list.append(item)
    new_series = pd.Series(no_neg_list)
    if len(new_series.value_counts().keys()) == 0:
        return pd.DataFrame()
    base_class = new_series.value_counts().keys()[0]
    dummies = pd.get_dummies(new_series,prefix=column.name)#+'_'+str(base_class),dummy_na=True)
    return dummies

In [25]:
def clean_numeric_column(column):
    # Don't process things that aren't actually columns
    if type(column) == pd.core.frame.DataFrame:
        return clean_numeric_column(column.iloc[:,0])
    # ensure all values are numeric -- or as many as possible
    try:
        col_numeric = column.astype(np.float32)
    except:
        col_replaced_numeric = []
        for item in column:
            try:
                col_replaced_numeric.append(np.float32(item))
            except:
                col_replaced_numeric.append(np.nan)
        col_numeric = pd.Series(col_replaced_numeric)
    # deal with negative values
    unique_negative_values = len(pd.Series([item for item in col_numeric if item < 0]).value_counts().keys())
    col_cleaned = []
    if unique_negative_values <= 5: # if there five or fewer unique negative values, remove them. This recognises cases where negative values are special markers
        for value in col_numeric:
            if value >= 0:
                col_cleaned.append(value)
            else:
                col_cleaned.append(np.nan)
        output = pd.Series(col_cleaned)
    else:
        output = col_numeric
    output.name = column.name
    return output

In [26]:
general_matrix_processed = pd.DataFrame()

In [27]:
numeric_columns = [col for col in general_matrix.columns if id_to_type[col] == 'Continuous' or id_to_type[col] == 'Integer']
categorical_columns = [col for col in general_matrix.columns if 'Categorical' in id_to_type[col]]

In [28]:
processed_numeric_columns = [clean_numeric_column(general_matrix[col]) for col in tqdm(numeric_columns)]
processed_categorical_columns = [clean_categorical_column_with_dummies(general_matrix[col]) for col in tqdm(categorical_columns)]

100%|████████████████████████████████████████| 124/124 [00:33<00:00,  3.67it/s]


In [29]:
for col in processed_numeric_columns:
    general_matrix_processed[feature_id_to_name(col.name)] = col

In [30]:
for df_dummies in processed_categorical_columns:
    for col in df_dummies.columns:
        general_matrix_processed[feature_id_to_name(col)] = df_dummies[col]

In [31]:
self_reported_matrix.columns = [feature_id_to_name('Self_Report_'+str(col)) for col in self_reported_matrix.columns]

In [32]:
icd_10_matrix.columns = [feature_id_to_name('ICD_10_'+str(col)) for col in icd_10_matrix.columns]

In [33]:
med_matrix.columns = [feature_id_to_name('Med_'+str(col)) for col in med_matrix.columns]

In [34]:
combined_matrix = pd.concat([general_matrix_processed, self_reported_matrix, icd_10_matrix, med_matrix],axis=1)

# Manual tidy up

In [35]:
combined_matrix['Number of live births_0'] = combined_matrix['Number of live births_0'].fillna(0)

In [36]:
combined_matrix['Cylindrical power (highest)'] = [np.nanmax([l,r]) for l,r in zip(combined_matrix['Cylindrical power (left)_0'], combined_matrix['Cylindrical power (right)_0'])]

In [37]:
combined_matrix['3mm asymmetry index (highest)'] = [np.nanmax([l,r]) for l,r in zip(combined_matrix['3mm asymmetry index (left)_0'], combined_matrix['3mm asymmetry index (right)_0'])]

In [38]:
combined_matrix['3mm regularity index (highest)'] = [np.nanmax([l,r]) for l,r in zip(combined_matrix['3mm regularity index (left)_0'], combined_matrix['3mm regularity index (right)_0'])]

In [39]:
combined_matrix['logMAR, final (highest)'] = [np.nanmax([l,r]) for l,r in zip(combined_matrix['logMAR, final (right)_0'], combined_matrix['logMAR, final (left)_0'])]

In [40]:
combined_matrix['Intra-ocular pressure, corneal-compensated (highest)'] = [np.nanmax([l,r]) for l,r in zip(combined_matrix[ 'Intra-ocular pressure, corneal-compensated (right)_0'], combined_matrix['Intra-ocular pressure, corneal-compensated (left)_0'])]

In [41]:
combined_matrix['Corneal hysteresis (lowest)'] = [np.nanmin([l,r]) for l,r in zip(combined_matrix['Corneal hysteresis (right)_0'], combined_matrix['Corneal hysteresis (left)_0'])]

In [42]:
combined_matrix['Corneal resistance factor (highest)'] =  [np.nanmax([l,r]) for l,r in zip(combined_matrix['Corneal resistance factor (right)_0'], combined_matrix['Corneal resistance factor (left)_0'])]

In [43]:
health_score_scaler = StandardScaler()

In [44]:
combined_matrix[['Health score (England)','Health score (Scotland)', 'Health score (Wales)']] = health_score_scaler.fit_transform(combined_matrix[['Health score (England)','Health score (Scotland)', 'Health score (Wales)']])

In [45]:
dump(health_score_scaler, '../glaucoma_project/Models/health_score_scaler.joblib')

['../glaucoma_project/Models/health_score_scaler.joblib']

In [46]:
combined_matrix['Health score'] = [e if not pd.isna(e) else (s if not pd.isna(s) else w) for e,s,w in zip(combined_matrix['Health score (England)'], combined_matrix['Health score (Scotland)'], combined_matrix['Health score (Wales)'])]

In [47]:
combined_matrix['Vertical cup to disc ratio (VCDR) (highest)'] = [np.nanmax([l,r]) for l,r in zip(combined_matrix['Vertical cup to disc ratio (VCDR) (left)_0'], combined_matrix['Vertical cup to disc ratio (VCDR) (right)_0'])]

In [48]:
combined_matrix['Average ganglion cell-inner plexiform layer thickness (lowest)'] = [np.nanmin([l,r]) for l,r in zip(combined_matrix[ 'Average ganglion cell-inner plexiform layer thickness (left)_0'], combined_matrix['Average ganglion cell-inner plexiform layer thickness (right)_0'])]

In [49]:
combined_matrix['Overall health rating_0'] = general_matrix['f.2178.0.0']

In [50]:
diabetes_diagnosis_cols = ['Diabetes diagnosed by doctor_0_1.0','diabetes'] + list([col for col in combined_matrix.columns if 'E1' in col])

In [51]:
def merge_columns(df, cols_to_merge):
    current_col = df[cols_to_merge[0]].fillna(0).astype(bool)
    for col in cols_to_merge:
        current_col = current_col | df[col].fillna(0).astype(bool)
    return current_col.astype(int)

In [52]:
combined_matrix['Diabetes Diagnosis'] = merge_columns(combined_matrix, diabetes_diagnosis_cols)

In [53]:
combined_matrix['Urban'] = merge_columns(combined_matrix, ['Home area population density - urban or rural_5.0','Home area population density - urban or rural_11.0'])

In [54]:
combined_matrix['FI2 : identify largest number_Incorrect'] = merge_columns(combined_matrix,  ['FI2 : identify largest number_1.0', 'FI2 : identify largest number_2.0',
                                                                                              'FI2 : identify largest number_4.0', 'FI2 : identify largest number_5.0'])

In [55]:
combined_matrix['Ethnic background_0_White/Other'] = merge_columns(combined_matrix, ['Ethnic background_0_1.0','Ethnic background_0_1001.0','Ethnic background_0_1002.0',
                                                                                     'Ethnic background_0_1003.0','Ethnic background_0_6.0','Ethnic background_0_2004.0','Ethnic background_0_2.0'])
combined_matrix['Ethnic background_0_Black'] = merge_columns(combined_matrix, ['Ethnic background_0_4.0','Ethnic background_0_4001.0','Ethnic background_0_4002.0',
                                                                               'Ethnic background_0_4003.0','Ethnic background_0_2001.0','Ethnic background_0_2002.0'])
combined_matrix['Ethnic background_0_Asian'] = merge_columns(combined_matrix, ['Ethnic background_0_3.0','Ethnic background_0_5.0','Ethnic background_0_3001.0',
 'Ethnic background_0_3002.0','Ethnic background_0_3003.0','Ethnic background_0_3004.0','Ethnic background_0_2003.0'])

In [56]:
combined_matrix['High Cholesterol Diagnosis'] = merge_columns(combined_matrix, ['high cholesterol','E78.0 Pure hypercholesterolaemia'])

In [57]:
arthritis_cols = list([col for col in combined_matrix.columns if 'M1' in col]) + ['osteoarthritis']

In [58]:
combined_matrix['Osteoarthritis Diagnosis'] = merge_columns(combined_matrix, arthritis_cols)

In [59]:
statins = [col for col in combined_matrix.columns if 'vastatin' in col]

In [60]:
combined_matrix['On Statin'] = merge_columns(combined_matrix, statins)

In [61]:
ppis = [col for col in combined_matrix.columns if 'prazole' in col] + ['Medication for pain relief, constipation, heartburn_0_5.0','Medication for pain relief, constipation, heartburn_1_5.0','Medication for pain relief, constipation, heartburn_2_5.0','Medication for pain relief, constipation, heartburn_3_5.0','Medication for pain relief, constipation, heartburn_4_5.0']

In [62]:
combined_matrix['On PPI'] = merge_columns(combined_matrix, ppis)

In [63]:
combined_matrix['Never eat sugar'] = merge_columns(combined_matrix, ['Never eat eggs, dairy, wheat, sugar_0_4.0','Never eat eggs, dairy, wheat, sugar_1_4.0',
                                                                    'Never eat eggs, dairy, wheat, sugar_2_4.0','Never eat eggs, dairy, wheat, sugar_3_4.0'])

In [64]:
never_eat_eggs_cols = [col for col in combined_matrix.columns if 'Never eat eggs, dairy, wheat, sugar' in col]

In [65]:
dental_problems_cols = [col for col in combined_matrix.columns if 'Mouth/teeth dental problems' in col]

In [66]:
combined_matrix['Bleeding gums'] = merge_columns(combined_matrix,['Mouth/teeth dental problems_0_3.0','Mouth/teeth dental problems_1_3.0','Mouth/teeth dental problems_2_3.0'])

In [67]:
combined_matrix['Dentures'] = merge_columns(combined_matrix,['Mouth/teeth dental problems_0_6.0','Mouth/teeth dental problems_1_6.0','Mouth/teeth dental problems_2_6.0',
                                                            'Mouth/teeth dental problems_3_6.0','Mouth/teeth dental problems_4_6.0','Mouth/teeth dental problems_5_6.0'])

In [68]:
vasc_problems_cols = [col for col in combined_matrix.columns if 'Vascular/heart problems' in col]

In [69]:
combined_matrix['Hypertension Diagnosis'] = merge_columns(combined_matrix,['Vascular/heart problems diagnosed by doctor_0_4.0','Vascular/heart problems diagnosed by doctor_1_4.0',
                                                                          'Vascular/heart problems diagnosed by doctor_2_4.0','Vascular/heart problems diagnosed by doctor_3_4.0',
                                                                           'hypertension','I10 Essential (primary) hypertension'])

In [70]:
analg_meds_cols = [col for col in combined_matrix.columns if 'Medication for pain relief, constipation, heartburn' in col]

In [71]:
combined_matrix['On Ibuprofen'] = merge_columns(combined_matrix,['ibuprofen', 'Medication for pain relief, constipation, heartburn_0_2.0','Medication for pain relief, constipation, heartburn_1_2.0'])

In [72]:
leisure_cols = [col for col in combined_matrix.columns if 'Leisure/social' in col]

In [73]:
combined_matrix['Sports club or gym'] = combined_matrix['Leisure/social activities_0_1.0']

In [74]:
combined_matrix['Religious group'] = merge_columns(combined_matrix, ['Leisure/social activities_0_3.0','Leisure/social activities_1_3.0','Leisure/social activities_2_3.0'])

In [75]:
ex_type_cols = [col for col in combined_matrix.columns if 'Types of physical activity' in col]

In [76]:
combined_matrix['Swimming/cycling/bowling/keep fit'] = merge_columns(combined_matrix,['Types of physical activity in last 4 weeks_0_2.0','Types of physical activity in last 4 weeks_1_2.0'])

In [77]:
supplement_cols = [col for col in combined_matrix.columns if 'Mineral and other dietary' in col] + ['glucosamine product','omega-3/fish oil supplement']

In [78]:
combined_matrix['On Fish Oil'] = merge_columns(combined_matrix,['Mineral and other dietary supplements_0_1.0','omega-3/fish oil supplement'])

In [79]:
combined_matrix['On Glucosamine'] = merge_columns(combined_matrix,['Mineral and other dietary supplements_0_2.0','Mineral and other dietary supplements_1_2.0','glucosamine product'])

In [80]:
illnesses_of_family_col = [col for col in combined_matrix.columns if 'Illnesses of' in col]

In [81]:
combined_matrix['Family history diabetes'] = merge_columns(combined_matrix,['Illnesses of father_0_9.0','Illnesses of father_1_9.0','Illnesses of father_2_9.0',
                                                                           'Illnesses of father_3_9.0','Illnesses of father_4_9.0','Illnesses of mother_0_9.0',
                                                                           'Illnesses of mother_1_9.0','Illnesses of mother_2_9.0','Illnesses of mother_3_9.0',
                                                                           'Illnesses of siblings_0_9.0','Illnesses of siblings_1_9.0','Illnesses of siblings_2_9.0',
                                                                           'Illnesses of siblings_3_9.0','Illnesses of siblings_4_9.0'])

In [82]:
combined_matrix['Family history hypertension'] = merge_columns(combined_matrix,['Illnesses of father_0_8.0','Illnesses of father_1_8.0','Illnesses of father_2_8.0','Illnesses of father_3_8.0','Illnesses of father_4_8.0',
                                                                                'Illnesses of mother_0_8.0','Illnesses of mother_1_8.0','Illnesses of mother_2_8.0','Illnesses of mother_3_8.0','Illnesses of mother_4_8.0',
                                                                                'Illnesses of siblings_0_8.0','Illnesses of siblings_1_8.0','Illnesses of siblings_2_8.0','Illnesses of siblings_3_8.0','Illnesses of siblings_4_8.0'])

In [83]:
combined_matrix['IBS Diagnosis'] = merge_columns(combined_matrix,['Ever diagnosed with IBS_1.0','irritable bowel syndrome','K58.0 Irritable bowel syndrome with diarrhoea','K58.9 Irritable bowel syndrome without diarrhoea'])

In [84]:
combined_matrix['Hiatus Hernia Diagnosis'] = merge_columns(combined_matrix,['K44.9 Diaphragmatic hernia without obstruction or gangrene','hiatus hernia'])

In [85]:
ibs_cols = [col for col in combined_matrix.columns if 'bowel' in col] + [ 'Ever diagnosed with IBS_0.0','Ever diagnosed with IBS_1.0']

In [86]:
cols_to_remove = ['Cylindrical power (left)_0','Cylindrical power (right)_0','3mm asymmetry index (left)_0','3mm asymmetry index (right)_0','3mm regularity index (left)_0',
                 '3mm regularity index (right)_0','logMAR, final (right)_0','logMAR, final (left)_0','Intra-ocular pressure, corneal-compensated (right)_0',
                 'Intra-ocular pressure, corneal-compensated (left)_0','Corneal hysteresis (right)_0','Corneal hysteresis (left)_0','Corneal resistance factor (right)_0',
                 'Corneal resistance factor (left)_0','Health score (England)','Health score (Scotland)','Health score (Wales)','Vertical cup to disc ratio (VCDR) (left)_0',
                 'Vertical cup to disc ratio (VCDR) (right)_0', 'Average ganglion cell-inner plexiform layer thickness (left)_0', 'Average ganglion cell-inner plexiform layer thickness (right)_0',
                  'Sex_0.0','Maternal smoking around birth_0_0.0','Miserableness_0_0.0','Overall health rating_0_1.0','Overall health rating_0_2.0','Overall health rating_0_3.0',
                  'Overall health rating_0_4.0','Wears glasses or contact lenses_0_0.0','Hearing difficulty/problems with background noise_0_0.0', 'Wheeze or whistling in the chest in last year_0_0.0',
                  'Chest pain or discomfort_0_0.0', 'Diabetes diagnosed by doctor_0_0.0','Ever taken oral contraceptive pill_0_0.0', 'Ever used hormone-replacement therapy (HRT)_0_0.0',
                  'Neck/shoulder pain for 3+ months_0_0.0', 'Knee pain for 3+ months_0_0.0', 'Ever highly irritable/argumentative for 2 days_0_0.0', 'Mouth/teeth dental problems_0_1.0',
                 'Mouth/teeth dental problems_0_2.0','Mouth/teeth dental problems_0_4.0', 'Mouth/teeth dental problems_0_5.0', 'Home area population density - urban or rural_1.0', 'Home area population density - urban or rural_2.0',
 'Home area population density - urban or rural_3.0', 'Home area population density - urban or rural_4.0','Home area population density - urban or rural_5.0',
 'Home area population density - urban or rural_6.0','Home area population density - urban or rural_7.0','Home area population density - urban or rural_8.0',
 'Home area population density - urban or rural_9.0','Home area population density - urban or rural_11.0','Home area population density - urban or rural_12.0',
 'Home area population density - urban or rural_13.0','Home area population density - urban or rural_14.0', 'Home area population density - urban or rural_16.0',
'Home area population density - urban or rural_17.0', 'Home area population density - urban or rural_18.0','FI2 : identify largest number_1.0', 'FI2 : identify largest number_2.0',
 'FI2 : identify largest number_3.0', 'FI2 : identify largest number_4.0', 'FI2 : identify largest number_5.0','Ever seen an un-real vision_0.0', 'Did your sleep change?_0.0',
 'Ethnic background_0_1.0','Ethnic background_0_2.0','Ethnic background_0_3.0','Ethnic background_0_4.0','Ethnic background_0_5.0','Ethnic background_0_6.0','Ethnic background_0_1001.0','Ethnic background_0_1002.0',
 'Ethnic background_0_1003.0','Ethnic background_0_2001.0','Ethnic background_0_2002.0','Ethnic background_0_2003.0','Ethnic background_0_2004.0','Ethnic background_0_3001.0',
 'Ethnic background_0_3002.0','Ethnic background_0_3003.0','Ethnic background_0_3004.0','Ethnic background_0_4001.0','Ethnic background_0_4002.0','Ethnic background_0_4003.0',
                  'Family history of IBS_0.0','high cholesterol','E78.0 Pure hypercholesterolaemia','hypertension','I10 Essential (primary) hypertension','ibuprofen','K44.9 Diaphragmatic hernia without obstruction or gangrene','hiatus hernia'
] + diabetes_diagnosis_cols + arthritis_cols + statins + ppis + never_eat_eggs_cols + dental_problems_cols + vasc_problems_cols + analg_meds_cols + leisure_cols + ex_type_cols + supplement_cols + illnesses_of_family_col + ibs_cols

In [87]:
new_set = [col for col in combined_matrix.columns if not col in cols_to_remove]

In [88]:
cleaned_matrix = combined_matrix[new_set]

In [89]:
cleaned_matrix.to_pickle(UKBB_PATH+'processed_data/Clinical Feature Selection/cleaned_matrix_with_selected_variables.pkl')

In [8]:
cleaned_matrix = pd.read_pickle(UKBB_PATH+'processed_data/Clinical Feature Selection/cleaned_matrix_with_selected_variables.pkl')

# Classification

In [1]:
classification_dict = {
 'Waist circumference_0': ['Biometrics','1b'],
 'Townsend deprivation index at recruitment': ['Demographics','1b'],
 'Number of days/week of vigorous physical activity 10+ minutes_0': ['Lifestyle','1b'],
 'Time spend outdoors in summer_0': ['Lifestyle','3'],
 'Time spent outdoors in winter_0': ['Lifestyle','3'],
 'Time spent watching television (TV)_0': ['Lifestyle','3'],
 'Time spent using computer_0': ['Lifestyle','3'],
 'Cooked vegetable intake_0': ['Diet','3'],
 'Fresh fruit intake_0': ['Diet','3'],
 'Dried fruit intake_0': ['Diet','3'],
 'Bread intake_0': ['Diet','3'],
 'Cereal intake_0': ['Diet','3'],
 'Tea intake_0': ['Diet','3'],
 'Average weekly spirits intake_0': ['Alcohol/drugs','3'],
 'Age started wearing glasses or contact lenses_0': ['Vision','1a'],
 'Frequency of solarium/sunlamp use_0': ['Lifestyle','3'],
 'Number of live births_0': ['Demographics','1b'],
 'Forced expiratory volume in 1-second (FEV1)_0': ['Misc Investigation','1b'],
 'Time to complete round_0': ['Misc Investigation','3'],
 'Pack years of smoking_0': ['Alcohol/drugs','1a'],
 'Duration to entering symbol choice_0': ['Misc Investigation','3'],
 'Forced vital capacity (FVC) Z-score': ['Misc Investigation','1b'],
 'Body mass index (BMI)_0': ['Biometrics','1a'],
 'Age when attended assessment centre_0': ['Demographics','1a'],
 'MET minutes per week for vigorous activity': ['Lifestyle','3'],
 'Particulate matter air pollution (pm10); 2007': ['Environment','3'],
 'Monocyte count_0': ['Bloods', '3'],
 'Microalbumin in urine_0': ['Misc Investigation','1b'],
 'Apolipoprotein B_0': ['Bloods', '3'],
 'Aspartate aminotransferase_0': ['Bloods', '1b'],
 'Cholesterol_0': ['Bloods', '1a'],
 'Creatinine_0': ['Bloods', '1b'],
 'C-reactive protein_0': ['Bloods', '1b'],
 'Cystatin C_0': ['Bloods', '3'],
 'Gamma glutamyltransferase_0': ['Bloods', '1b'],
 'Glucose_0': ['Bloods', '1b'],
 'Glycated haemoglobin (HbA1c)_0': ['Bloods', '1a'],
 'HDL cholesterol_0': ['Bloods', '1a'],
 'IGF-1_0': ['Bloods', '3'],
 'LDL direct_0': ['Bloods', '1a'],
 'Oestradiol_0': ['Bloods', '3'],
 'Sex_1.0': ['Demographics', '1a'],
 'Average total household income before tax_0_1.0': ['Demographics', '3'],
 'Average total household income before tax_0_2.0': [None, '3'],
 'Average total household income before tax_0_3.0': [None, '3'],
 'Average total household income before tax_0_4.0': [None, '3'],
 'Average total household income before tax_0_5.0': [None, '3'],
 'Usual walking pace_0_1.0': ['Lifestyle', '3'],
 'Usual walking pace_0_2.0': [None, '3'],
 'Usual walking pace_0_3.0': [None, '3'],
 'Frequency of stair climbing in last 4 weeks_0_0.0': ['Lifestyle', '3'],
 'Frequency of stair climbing in last 4 weeks_0_1.0': [None, '3'],
 'Frequency of stair climbing in last 4 weeks_0_2.0': [None, '3'],
 'Frequency of stair climbing in last 4 weeks_0_3.0': [None, '3'],
 'Frequency of stair climbing in last 4 weeks_0_4.0': [None, '3'],
 'Frequency of stair climbing in last 4 weeks_0_5.0': [None, '3'],
 'Length of mobile phone use_0_0.0': ['Lifestyle', '3'],
 'Length of mobile phone use_0_1.0': [None, '3'],
 'Length of mobile phone use_0_2.0': [None, '3'],
 'Length of mobile phone use_0_3.0': [None, '3'],
 'Length of mobile phone use_0_4.0': [None, '3'],
 'Weekly usage of mobile phone in last 3 months_0_0.0': ['Lifestyle', '3'],
 'Weekly usage of mobile phone in last 3 months_0_1.0': [None, '3'],
 'Weekly usage of mobile phone in last 3 months_0_2.0': [None, '3'],
 'Weekly usage of mobile phone in last 3 months_0_3.0': [None, '3'],
 'Weekly usage of mobile phone in last 3 months_0_4.0': [None, '3'],
 'Weekly usage of mobile phone in last 3 months_0_5.0': [None, '3'],
 'Getting up in morning_0_1.0': ['Sleep', '3'],
 'Getting up in morning_0_2.0': [None, '3'],
 'Getting up in morning_0_3.0': [None, '3'],
 'Getting up in morning_0_4.0': [None, '3'],
 'Nap during day_0_1.0': ['Sleep', '3'],
 'Nap during day_0_2.0': [None, '3'],
 'Nap during day_0_3.0': [None, '3'],
 'Sleeplessness / insomnia_0_1.0': ['Sleep', '1b'],
 'Sleeplessness / insomnia_0_2.0': [None, '1b'],
 'Sleeplessness / insomnia_0_3.0': [None, '1b'],
 'Daytime dozing / sleeping (narcolepsy)_0_0.0': ['Sleep', '1b'],
 'Daytime dozing / sleeping (narcolepsy)_0_1.0': [None, '1b'],
 'Daytime dozing / sleeping (narcolepsy)_0_2.0': [None, '1b'],
 'Daytime dozing / sleeping (narcolepsy)_0_3.0': [None, '1b'],
 'Current tobacco smoking_0_0.0': ['Alcohol/drugs', '1b'],
 'Current tobacco smoking_0_1.0': [None, '1b'],
 'Current tobacco smoking_0_2.0': [None, '1b'],
 'Past tobacco smoking_0_1.0': ['Alcohol/drugs', '3'],
 'Past tobacco smoking_0_2.0': [None, '3'],
 'Past tobacco smoking_0_3.0': [None, '3'],
 'Past tobacco smoking_0_4.0': [None, '3'],
 'Smoking/smokers in household_0_0.0': ['Alcohol/drugs', '3'],
 'Smoking/smokers in household_0_1.0': [None, '3'],
 'Smoking/smokers in household_0_2.0': [None, '3'],
 'Oily fish intake_0_0.0': ['Diet','3'],
 'Oily fish intake_0_1.0': [None,'3'],
 'Oily fish intake_0_2.0': [None,'3'],
 'Oily fish intake_0_3.0': [None,'3'],
 'Oily fish intake_0_4.0': [None,'3'],
 'Oily fish intake_0_5.0': [None,'3'],
 'Non-oily fish intake_0_0.0': ['Diet','3'],
 'Non-oily fish intake_0_1.0': [None,'3'],
 'Non-oily fish intake_0_2.0': [None,'3'],
 'Non-oily fish intake_0_3.0': [None,'3'],
 'Non-oily fish intake_0_4.0': [None,'3'],
 'Non-oily fish intake_0_5.0': [None,'3'],
 'Poultry intake_0_0.0': ['Diet','3'],
 'Poultry intake_0_1.0': [None,'3'],
 'Poultry intake_0_2.0': [None,'3'],
 'Poultry intake_0_3.0': [None,'3'],
 'Poultry intake_0_4.0': [None,'3'],
 'Poultry intake_0_5.0': [None,'3'],
 'Lamb/mutton intake_0_0.0': ['Diet','3'],
 'Lamb/mutton intake_0_1.0': [None,'3'],
 'Lamb/mutton intake_0_2.0': [None,'3'],
 'Lamb/mutton intake_0_3.0': [None,'3'],
 'Lamb/mutton intake_0_4.0': [None,'3'],
 'Lamb/mutton intake_0_5.0': [None,'3'],
 'Pork intake_0_0.0': ['Diet','3'],
 'Pork intake_0_1.0': [None,'3'],
 'Pork intake_0_2.0': [None,'3'],
 'Pork intake_0_3.0': [None,'3'],
 'Pork intake_0_4.0': [None,'3'],
 'Pork intake_0_5.0': [None,'3'],
 'Major dietary changes in the last 5 years_0_0.0': ['Diet','3'],
 'Major dietary changes in the last 5 years_0_1.0': [None,'3'],
 'Major dietary changes in the last 5 years_0_2.0': [None,'3'],
 'Alcohol intake frequency._0_1.0': ['Alcohol/drugs','1b'],
 'Alcohol intake frequency._0_2.0': [None,'1b'],
 'Alcohol intake frequency._0_3.0': [None,'1b'],
 'Alcohol intake frequency._0_4.0': [None,'1b'],
 'Alcohol intake frequency._0_5.0': [None,'1b'],
 'Alcohol intake frequency._0_6.0': [None,'1b'],
 'Maternal smoking around birth_0_1.0': ['Early life','3'],
 'Miserableness_0_1.0': ['Mental health','1b'],
 'Frequency of depressed mood in last 2 weeks_0_1.0': ['Mental health', '1b'],
 'Frequency of depressed mood in last 2 weeks_0_2.0': [None,'1b'],
 'Frequency of depressed mood in last 2 weeks_0_3.0': [None,'1b'],
 'Frequency of depressed mood in last 2 weeks_0_4.0': [None,'1b'],
 'Frequency of tiredness / lethargy in last 2 weeks_0_1.0': ['Mental health', '1b'],
 'Frequency of tiredness / lethargy in last 2 weeks_0_2.0': [None,'1b'],
 'Frequency of tiredness / lethargy in last 2 weeks_0_3.0': [None,'1b'],
 'Frequency of tiredness / lethargy in last 2 weeks_0_4.0': [None,'1b'],
 'Wears glasses or contact lenses_0_1.0': ['Vision','1a'],
 'Hearing difficulty/problems_0_0.0': ['Hearing','1b'],
 'Hearing difficulty/problems_0_1.0': [None,'1b'],
 'Hearing difficulty/problems_0_99.0': [None,'1b'],
 'Hearing difficulty/problems with background noise_0_1.0': ['Hearing','1b'],
 'Use of sun/uv protection_0_1.0': ['Lifestyle','3'],
 'Use of sun/uv protection_0_2.0': [None,'3'],
 'Use of sun/uv protection_0_3.0': [None,'3'],
 'Use of sun/uv protection_0_4.0': [None,'3'],
 'Use of sun/uv protection_0_5.0': [None,'3'],
 'Falls in the last year_0_1.0': ['Symptom','1b'],
 'Falls in the last year_0_2.0': [None,'1b'],
 'Falls in the last year_0_3.0': [None,'1b'],
 'Wheeze or whistling in the chest in last year_0_1.0': ['Symptom','1b'],
 'Chest pain or discomfort_0_1.0': ['Symptom','1b'],
 'Relative age voice broke_0_1.0': ['Early life','3'],
 'Relative age voice broke_0_2.0': [None,'3'],
 'Relative age voice broke_0_3.0': [None,'3'],
 'Had menopause_0_0.0': ['Demographics','1b'],
 'Had menopause_0_1.0': [None,'1b'],
 'Had menopause_0_2.0': [None,'1b'],
 'Had menopause_0_3.0': [None,'1b'],
 'Ever taken oral contraceptive pill_0_1.0': ['Medication','1b'],
 'Ever used hormone-replacement therapy (HRT)_0_1.0': ['Medication','1b'],
 'Neck/shoulder pain for 3+ months_0_1.0': ['Symptom', '1b'],
 'Frequency of other exercises in last 4 weeks_0_1.0': ['Lifestyle','3'],
 'Frequency of other exercises in last 4 weeks_0_2.0': [None,'3'],
 'Frequency of other exercises in last 4 weeks_0_3.0': [None,'3'],
 'Frequency of other exercises in last 4 weeks_0_4.0': [None,'3'],
 'Frequency of other exercises in last 4 weeks_0_5.0': [None,'3'],
 'Frequency of other exercises in last 4 weeks_0_6.0': [None,'3'],
 'Duration of other exercises_0_1.0': ['Lifestyle','3'],
 'Duration of other exercises_0_2.0': [None,'3'],
 'Duration of other exercises_0_3.0': [None,'3'],
 'Duration of other exercises_0_4.0': [None,'3'],
 'Duration of other exercises_0_5.0': [None,'3'],
 'Duration of other exercises_0_6.0': [None,'3'],
 'Duration of other exercises_0_7.0': [None,'3'],
 'Knee pain for 3+ months_0_1.0': ['Symptom', '1b'],
 'Ever highly irritable/argumentative for 2 days_0_1.0': ['Mental health', '1b'],
 'Smoking status_0_0.0': ['Alcohol/drugs', '1a'],
 'Smoking status_0_1.0': [None, '1a'],
 'Smoking status_0_2.0': [None, '1a'],
 'Ever seen an un-real vision_1.0': ['Mental health', '1b'],
 'Recent easy annoyance or irritability_1.0': ['Mental health', '1b'],
 'Recent easy annoyance or irritability_2.0': [None,'1b'],
 'Recent easy annoyance or irritability_3.0': [None,'1b'],
 'Recent easy annoyance or irritability_4.0': [None,'1b'],
 'Recent trouble concentrating on things_1.0': ['Mental health','1b'],
 'Recent trouble concentrating on things_2.0': [None,'1b'],
 'Recent trouble concentrating on things_3.0': [None,'1b'],
 'Recent trouble concentrating on things_4.0': [None,'1b'],
 'Recent feelings of depression_1.0': ['Mental health', '1b'],
 'Recent feelings of depression_2.0': [None,'1b'],
 'Recent feelings of depression_3.0': [None,'1b'],
 'Recent feelings of depression_4.0': [None,'1b'],
 'Recent feelings of foreboding_1.0': ['Mental health', '1b'],
 'Recent feelings of foreboding_2.0': [None,'1b'],
 'Recent feelings of foreboding_3.0': [None,'1b'],
 'Recent feelings of foreboding_4.0': [None,'1b'],
 'Recent feelings of tiredness or low energy_1.0': ['Mental health', '1b'],
 'Recent feelings of tiredness or low energy_2.0': [None,'1b'],
 'Recent feelings of tiredness or low energy_3.0': [None,'1b'],
 'Recent feelings of tiredness or low energy_4.0': [None,'1b'],
 'Did your sleep change?_1.0': ['Mental health','1b'],
 'Family history of IBS_1.0': ['Family history','1b'],
 'aspirin': ['Medication','1b'],
 'Cylindrical power (highest)': ['Vision','2'],
 '3mm asymmetry index (highest)': ['Vision','2'],
 '3mm regularity index (highest)': ['Vision','2'],
 'logMAR, final (highest)': ['Vision','2'],
 'Intra-ocular pressure, corneal-compensated (highest)': ['Vision','2'],
 'Corneal hysteresis (lowest)': ['Vision','2'],
 'Corneal resistance factor (highest)': ['Vision','2'],
 'Health score': ['Health ratings','3'],
 'Vertical cup to disc ratio (VCDR) (highest)': ['Vision','2'],
 'Average ganglion cell-inner plexiform layer thickness (lowest)': ['Vision','2'],
 'Overall health rating_0': ['Health ratings','3'],
 'Diabetes Diagnosis': ['Diagnosis','1a'],
 'Urban':['Demographics','1b'],
 'FI2 : identify largest number_Incorrect':['Misc Investigation','3'],
 'Ethnic background_0_White/Other':['Demographics','1a'],
 'Ethnic background_0_Black':[None,'1a'],
 'Ethnic background_0_Asian':[None,'1a'],
 'High Cholesterol Diagnosis': ['Diagnosis','1a'],
 'Osteoarthritis Diagnosis': ['Diagnosis','1b'],
 'On Statin': ['Medication','1a'],
 'On PPI': ['Medication','1b'],
 'Never eat sugar': ['Diet','3'],
 'Bleeding gums': ['Symptom','1b'],
 'Dentures': ['Diagnosis','1b'],
 'Hypertension Diagnosis': ['Diagnosis','1a'],
 'On Ibuprofen': ['Medication','1b'],
 'Sports club or gym': ['Lifestyle','3'],
 'Religious group': ['Lifestyle','3'],
 'Swimming/cycling/bowling/keep fit': ['Lifestyle','3'],
 'On Fish Oil': ['Medication','1b'],
 'On Glucosamine': ['Medication','1b'],
 'Family history diabetes': ['Family history','1b'],
 'Family history hypertension': ['Family history','1b'],
 'IBS Diagnosis': ['Diagnosis','1b'],
 'Hiatus Hernia Diagnosis': ['Diagnosis','1b']}

In [6]:
dump(classification_dict, UKBB_PATH+'processed_data/classification_dict.joblib')

['../glaucoma_project/UKBB_Data/processed_data/classification_dict.joblib']

In [3]:
classification_dict = load(UKBB_PATH+'processed_data/classification_dict.joblib')

In [4]:
len(classification_dict)

242

In [11]:
df_selected_field_data = pd.DataFrame()

In [12]:
df_selected_field_data['Field Name'] = cleaned_matrix.columns

In [13]:
df_selected_field_data['Field Type'] = df_selected_field_data['Field Name'].apply(lambda x: classification_dict[x][0])

In [14]:
df_selected_field_data['Model'] = df_selected_field_data['Field Name'].apply(lambda x: classification_dict[x][1])

In [15]:
df_selected_field_data = df_selected_field_data.dropna()

In [16]:
df_selected_field_data['Field Type'].value_counts()

Lifestyle             17
Bloods                14
Diet                  13
Mental health         11
Vision                11
Medication             8
Demographics           8
Diagnosis              7
Alcohol/drugs          7
Misc Investigation     6
Symptom                6
Sleep                  4
Family history         3
Hearing                2
Early life             2
Health ratings         2
Biometrics             2
Environment            1
Name: Field Type, dtype: int64

In [21]:
df_selected_field_data['Feature Set'] = df_selected_field_data['Model'].apply(lambda x: {'1a':'A', '1b':'B', '2':'C', '3':'D'}[x])

In [30]:
df_selected_field_data.sort_values(['Field Type','Model'], ascending=[True,True]).to_csv('chosen_fields_table.csv')

In [23]:
df_selected_field_data[df_selected_field_data['Field Type'] == 'Lifestyle']

,Field Name,Field Type,Model,Feature Set
2,Number of days/week of vigorous physical activ...,Lifestyle,1b,B
3,Time spend outdoors in summer_0,Lifestyle,3,D
4,Time spent outdoors in winter_0,Lifestyle,3,D
5,Time spent watching television (TV)_0,Lifestyle,3,D
6,Time spent using computer_0,Lifestyle,3,D
15,Frequency of solarium/sunlamp use_0,Lifestyle,3,D
24,MET minutes per week for vigorous activity,Lifestyle,3,D
47,Usual walking pace_0_1.0,Lifestyle,3,D
50,Frequency of stair climbing in last 4 weeks_0_0.0,Lifestyle,3,D
56,Length of mobile phone use_0_0.0,Lifestyle,3,D


In [24]:
df_selected_field_data[df_selected_field_data['Field Type'] == 'Bloods']

,Field Name,Field Type,Model,Feature Set
26,Monocyte count_0,Bloods,3,D
28,Apolipoprotein B_0,Bloods,3,D
29,Aspartate aminotransferase_0,Bloods,1b,B
30,Cholesterol_0,Bloods,1a,A
31,Creatinine_0,Bloods,1b,B
32,C-reactive protein_0,Bloods,1b,B
33,Cystatin C_0,Bloods,3,D
34,Gamma glutamyltransferase_0,Bloods,1b,B
35,Glucose_0,Bloods,1b,B
36,Glycated haemoglobin (HbA1c)_0,Bloods,1a,A


In [25]:
df_selected_field_data[df_selected_field_data['Field Type'] == 'Diet']

,Field Name,Field Type,Model,Feature Set
7,Cooked vegetable intake_0,Diet,3,D
8,Fresh fruit intake_0,Diet,3,D
9,Dried fruit intake_0,Diet,3,D
10,Bread intake_0,Diet,3,D
11,Cereal intake_0,Diet,3,D
12,Tea intake_0,Diet,3,D
91,Oily fish intake_0_0.0,Diet,3,D
97,Non-oily fish intake_0_0.0,Diet,3,D
103,Poultry intake_0_0.0,Diet,3,D
109,Lamb/mutton intake_0_0.0,Diet,3,D


In [26]:
df_selected_field_data[df_selected_field_data['Field Type'] == 'Vision']

,Field Name,Field Type,Model,Feature Set
14,Age started wearing glasses or contact lenses_0,Vision,1a,A
140,Wears glasses or contact lenses_0_1.0,Vision,1a,A
207,Cylindrical power (highest),Vision,2,C
208,3mm asymmetry index (highest),Vision,2,C
209,3mm regularity index (highest),Vision,2,C
210,"logMAR, final (highest)",Vision,2,C
211,"Intra-ocular pressure, corneal-compensated (hi...",Vision,2,C
212,Corneal hysteresis (lowest),Vision,2,C
213,Corneal resistance factor (highest),Vision,2,C
215,Vertical cup to disc ratio (VCDR) (highest),Vision,2,C


In [27]:
df_selected_field_data[df_selected_field_data['Field Type'] == 'Mental health']

,Field Name,Field Type,Model,Feature Set
131,Miserableness_0_1.0,Mental health,1b,B
132,Frequency of depressed mood in last 2 weeks_0_1.0,Mental health,1b,B
136,Frequency of tiredness / lethargy in last 2 we...,Mental health,1b,B
179,Ever highly irritable/argumentative for 2 days...,Mental health,1b,B
183,Ever seen an un-real vision_1.0,Mental health,1b,B
184,Recent easy annoyance or irritability_1.0,Mental health,1b,B
188,Recent trouble concentrating on things_1.0,Mental health,1b,B
192,Recent feelings of depression_1.0,Mental health,1b,B
196,Recent feelings of foreboding_1.0,Mental health,1b,B
200,Recent feelings of tiredness or low energy_1.0,Mental health,1b,B


In [28]:
df_selected_field_data[df_selected_field_data['Field Type'] == 'Demographics']

,Field Name,Field Type,Model,Feature Set
1,Townsend deprivation index at recruitment,Demographics,1b,B
16,Number of live births_0,Demographics,1b,B
23,Age when attended assessment centre_0,Demographics,1a,A
41,Sex_1.0,Demographics,1a,A
42,Average total household income before tax_0_1.0,Demographics,3,D
158,Had menopause_0_0.0,Demographics,1b,B
219,Urban,Demographics,1b,B
221,Ethnic background_0_White/Other,Demographics,1a,A


In [29]:
df_selected_field_data[df_selected_field_data['Field Type'] == 'Diagnosis']

,Field Name,Field Type,Model,Feature Set
218,Diabetes Diagnosis,Diagnosis,1a,A
224,High Cholesterol Diagnosis,Diagnosis,1a,A
225,Osteoarthritis Diagnosis,Diagnosis,1b,B
230,Dentures,Diagnosis,1b,B
231,Hypertension Diagnosis,Diagnosis,1a,A
240,IBS Diagnosis,Diagnosis,1b,B
241,Hiatus Hernia Diagnosis,Diagnosis,1b,B


In [130]:
df_selected_field_data[df_selected_field_data['Field Type'] == 'Medication']

,Field Name,Field Type,Model
162,Ever taken oral contraceptive pill_0_1.0,Medication,1b
163,Ever used hormone-replacement therapy (HRT)_0_1.0,Medication,1b
206,aspirin,Medication,1b
226,On Statin,Medication,1a
227,On PPI,Medication,1b
232,On Ibuprofen,Medication,1b
236,On Fish Oil,Medication,1b
237,On Glucosamine,Medication,1b


In [131]:
df_selected_field_data[df_selected_field_data['Field Type'] == 'Misc Investigation']

,Field Name,Field Type,Model
17,Forced expiratory volume in 1-second (FEV1)_0,Misc Investigation,1b
18,Time to complete round_0,Misc Investigation,3
20,Duration to entering symbol choice_0,Misc Investigation,3
21,Forced vital capacity (FVC) Z-score,Misc Investigation,1b
27,Microalbumin in urine_0,Misc Investigation,1b
220,FI2 : identify largest number_Incorrect,Misc Investigation,3


In [132]:
df_selected_field_data[df_selected_field_data['Field Type'] == 'Alcohol/drugs']

,Field Name,Field Type,Model
13,Average weekly spirits intake_0,Alcohol/drugs,3
19,Pack years of smoking_0,Alcohol/drugs,1a
81,Current tobacco smoking_0_0.0,Alcohol/drugs,1b
84,Past tobacco smoking_0_1.0,Alcohol/drugs,3
88,Smoking/smokers in household_0_0.0,Alcohol/drugs,3
124,Alcohol intake frequency._0_1.0,Alcohol/drugs,1b
180,Smoking status_0_0.0,Alcohol/drugs,1a


In [133]:
df_selected_field_data[df_selected_field_data['Field Type'] == 'Symptom']

,Field Name,Field Type,Model
150,Falls in the last year_0_1.0,Symptom,1b
153,Wheeze or whistling in the chest in last year_...,Symptom,1b
154,Chest pain or discomfort_0_1.0,Symptom,1b
164,Neck/shoulder pain for 3+ months_0_1.0,Symptom,1b
178,Knee pain for 3+ months_0_1.0,Symptom,1b
229,Bleeding gums,Symptom,1b


In [134]:
df_selected_field_data[df_selected_field_data['Field Type'] == 'Sleep']

,Field Name,Field Type,Model
67,Getting up in morning_0_1.0,Sleep,3
71,Nap during day_0_1.0,Sleep,3
74,Sleeplessness / insomnia_0_1.0,Sleep,1b
77,Daytime dozing / sleeping (narcolepsy)_0_0.0,Sleep,1b


In [135]:
df_selected_field_data[df_selected_field_data['Field Type'] == 'Family history']

,Field Name,Field Type,Model
205,Family history of IBS_1.0,Family history,1b
238,Family history diabetes,Family history,1b
239,Family history hypertension,Family history,1b


In [136]:
df_selected_field_data[df_selected_field_data['Field Type'] == 'Hearing']

,Field Name,Field Type,Model
141,Hearing difficulty/problems_0_0.0,Hearing,1b
144,Hearing difficulty/problems with background no...,Hearing,1b


In [137]:
df_selected_field_data[df_selected_field_data['Field Type'] == 'Early life']

,Field Name,Field Type,Model
130,Maternal smoking around birth_0_1.0,Early life,3
155,Relative age voice broke_0_1.0,Early life,3


In [138]:
df_selected_field_data[df_selected_field_data['Field Type'] == 'Health ratings']

,Field Name,Field Type,Model
214,Health score,Health ratings,3
217,Overall health rating_0,Health ratings,3


In [139]:
df_selected_field_data[df_selected_field_data['Field Type'] == 'Biometrics']

,Field Name,Field Type,Model
0,Waist circumference_0,Biometrics,1b
22,Body mass index (BMI)_0,Biometrics,1a


In [140]:
df_selected_field_data[df_selected_field_data['Field Type'] == 'Environment']

,Field Name,Field Type,Model
25,Particulate matter air pollution (pm10); 2007,Environment,3


# Filtering

In [117]:
df_glaucoma_data = pd.read_pickle(UKBB_PATH+'processed_data/Clinical Feature Selection/glaucoma_data_checkpoint.pkl')

In [118]:
filter_series = (~df_glaucoma_data['Total Glaucoma'] | (df_glaucoma_data['Diagnosis Year'] > 2010))

In [119]:
filtered_matrix = cleaned_matrix[filter_series]

# Imputation

In [120]:
smp_imputer = SimpleImputer(strategy='mean')

In [121]:
imputed_matrix = pd.DataFrame(smp_imputer.fit_transform(filtered_matrix), columns=filtered_matrix.columns)

In [122]:
dump(smp_imputer, '../glaucoma_project/Models/smp_imputer_clinical_data.joblib')

['../glaucoma_project/Models/smp_imputer_clinical_data.joblib']

# Scaling

In [123]:
std_scaler = StandardScaler()

In [124]:
columns_to_scale = []

In [125]:
for col in tqdm(imputed_matrix.columns):
    if len(imputed_matrix[col].value_counts().keys()) > 2:
        columns_to_scale.append(col)

100%|███████████████████████████████████████| 242/242 [00:01<00:00, 240.61it/s]


In [126]:
scaled_cols = pd.DataFrame(std_scaler.fit_transform(imputed_matrix[columns_to_scale]), columns=columns_to_scale)

In [127]:
dump(std_scaler, '../glaucoma_project/Models/std_scaler_clinical_data.joblib')

['../glaucoma_project/Models/std_scaler_clinical_data.joblib']

In [128]:
imputed_matrix[columns_to_scale] = scaled_cols

In [129]:
imputed_matrix

,Waist circumference_0,Townsend deprivation index at recruitment,Number of days/week of vigorous physical activity 10+ minutes_0,Time spend outdoors in summer_0,Time spent outdoors in winter_0,Time spent watching television (TV)_0,Time spent using computer_0,Cooked vegetable intake_0,Fresh fruit intake_0,Dried fruit intake_0,...,On Ibuprofen,Sports club or gym,Religious group,Swimming/cycling/bowling/keep fit,On Fish Oil,On Glucosamine,Family history diabetes,Family history hypertension,IBS Diagnosis,Hiatus Hernia Diagnosis
0,-0.911800,-1.118830,-4.412120e-01,0.929051,-1.454543e-01,1.950884,0.000000,-0.942502,-0.818550,-5.055417e-01,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
1,-1.209223,0.084647,-4.412120e-01,0.478434,-7.553898e-01,-0.573127,-0.178616,-0.412012,-0.184510,-5.055417e-01,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.541941,-1.556288,-4.412120e-01,-0.422799,-1.454543e-01,-0.573127,-0.178616,-0.942502,-0.818550,6.851648e-02,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,1.021451,0.072867,-9.660884e-01,-1.324032,-2.708658e-16,-0.573127,-0.178616,-0.412012,0.449530,-5.055417e-01,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.019530,-0.901135,-4.412120e-01,-0.422799,-7.553898e-01,1.319881,0.000000,0.118478,11.228209,5.235040e+00,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445154,1.095807,-0.567292,-4.412120e-01,-0.422799,4.644813e-01,-1.204130,0.000000,0.648969,-0.818550,-6.373327e-17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
445155,-1.357934,0.955716,-9.660884e-01,-0.422799,-7.553898e-01,-1.204130,-0.178616,0.648969,-0.818550,6.851648e-02,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
445156,0.798383,-0.769106,-2.330919e-16,0.000000,-2.708658e-16,1.319881,-0.178616,-0.412012,1.083570,-6.373327e-17,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
445157,0.129181,-0.993041,8.366436e-02,-0.873415,-7.553898e-01,0.057876,0.000000,-0.412012,-0.184510,-5.055417e-01,...,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


In [130]:
glaucoma_diagnosis = df_glaucoma_data[filter_series]['Total Glaucoma'].astype(int).reset_index(drop=True)

In [131]:
imputed_matrix.to_pickle(UKBB_PATH+'processed_data/imputed_matrix_selected_clinical_features.pkl')
glaucoma_diagnosis.to_pickle(UKBB_PATH+'processed_data/glaucoma_diagnosis.pkl')

# Split into train and val for initial model selection

In [4]:
imputed_matrix = pd.read_pickle(UKBB_PATH+'processed_data/imputed_matrix_selected_clinical_features.pkl')
glaucoma_diagnosis = pd.read_pickle(UKBB_PATH+'processed_data/glaucoma_diagnosis.pkl')

In [4]:
glaucoma_diagnosis = pd.read_pickle(UKBB_PATH+'processed_data/glaucoma_diagnosis.pkl')

In [5]:
glaucoma_diagnosis.value_counts()

0    439661
1      5498
Name: Total Glaucoma, dtype: int64

In [6]:
train_idx, val_idx = train_test_split(range(len(glaucoma_diagnosis)),random_state=0,test_size=0.1)

In [7]:
X_train = imputed_matrix.iloc[train_idx].reset_index(drop=True)
y_train = glaucoma_diagnosis.iloc[train_idx].reset_index(drop=True)
X_val = imputed_matrix.iloc[val_idx].reset_index(drop=True)
y_val = glaucoma_diagnosis.iloc[val_idx].reset_index(drop=True)

# Models

In [14]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestRegressor, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
import xgboost as xgb

In [15]:
from sklearn.tree import DecisionTreeClassifier

In [16]:
from sklearn.metrics import roc_curve, accuracy_score, confusion_matrix

In [17]:
def specificity(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn+fp)
    return specificity

In [18]:
def sensitivity(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp+fn)
    return sensitivity

In [19]:
def cutoff_youdens_j(fpr,tpr,thresholds):
    j_scores = tpr-fpr
    j_ordered = sorted(zip(j_scores,thresholds))
    return j_ordered[-1][1]

In [20]:
def evaluate_model(y_true, y_pred_proba):
    if len(y_pred_proba.shape) == 2:
        probs = y_pred_proba[:,1]
    else:
        probs = y_pred_proba
    auc = roc_auc_score(y_true,probs)
    print('AUC',auc)
    fpr, tpr, thresholds = roc_curve(y_true, probs)
    yj_threshold = cutoff_youdens_j(fpr, tpr, thresholds)
    print('Youden\'s J threshold',yj_threshold)
    acc = accuracy_score(y_true,(probs > yj_threshold).astype(int))
    print('Accuracy',acc)
    sens = sensitivity(y_true,(probs > yj_threshold).astype(int))
    print('Sensitivity',sens)
    spec = specificity(y_true,(probs > yj_threshold).astype(int))
    print('Specificity',spec)
    return auc,yj_threshold,acc,sens,spec

# Model 0: Age-only model

In [11]:
age_clf = LogisticRegression(class_weight='balanced')

In [12]:
age_clf.fit(X_train[['Age when attended assessment centre_0']], y_train)

LogisticRegression(class_weight='balanced')

In [21]:
y_train_pred_proba = age_clf.predict_proba(X_train[['Age when attended assessment centre_0']])
age_train_stats = evaluate_model(y_train, y_train_pred_proba)

AUC 0.6908355935233436
Youden's J threshold 0.49480422517221134
Accuracy 0.5764159114223885
Sensitivity 0.7098941368078175
Specificity 0.5747591166726893


In [22]:
y_val_pred_proba = age_clf.predict_proba(X_val[['Age when attended assessment centre_0']])
age_val_stats = evaluate_model(y_val,y_val_pred_proba)

AUC 0.7049761915675652
Youden's J threshold 0.49480422517221134
Accuracy 0.5742429688201994
Sensitivity 0.7235494880546075
Specificity 0.5722513089005236


In [24]:
dump(age_clf, '../glaucoma_project/Models/age_model.joblib')

['../glaucoma_project/Models/age_model.joblib']

In [158]:
models = [LogisticRegression(class_weight='balanced',n_jobs=-1, max_iter=300),
SGDClassifier(class_weight='balanced',n_jobs=-1, random_state=42,loss='modified_huber'),
RandomForestClassifier(class_weight='balanced',max_depth=10, random_state=42, n_jobs=-1),
AdaBoostClassifier(DecisionTreeClassifier(class_weight='balanced',max_depth=10,random_state=42)),
GradientBoostingClassifier(random_state=42),
BaggingClassifier(DecisionTreeClassifier(class_weight='balanced',max_depth=10,random_state=42),n_jobs=-1,random_state=42),
LinearSVC(class_weight='balanced',random_state=42),
MLPClassifier(random_state=42),
KNeighborsClassifier(n_jobs=-1),
LinearDiscriminantAnalysis(),
QuadraticDiscriminantAnalysis()]

In [159]:
model_names = ['LogisticRegression',
               'SGDClassifier',
               'RandomForestClassifier',
               'AdaBoostClassifier',
               'GradientBoostingClassifier',
               'BaggingClassifier',
               'LinearSVC',
               'MLPClassifier',
               'KNeighborsClassifier',
               'LinearDiscriminantAnalysis',
               'QuadraticDiscriminantAnalysis'
]

In [197]:
X_train_1a = X_train[[col for col in classification_dict.keys() if classification_dict[col][1] == '1a']]
X_val_1a = X_val[[col for col in classification_dict.keys() if classification_dict[col][1] == '1a']]

In [174]:
for model, model_name in zip(models, model_names):
    print('Training',model_name)
    t0 = time.time()
    model.fit(X_train_1a,y_train)
    print('Training took',time.time()-t0)
    dump(model,'../glaucoma_project/Models/'+model_name+'_clinical_1a_model.joblib')

Training LogisticRegression
Training took 11.503736019134521
Training SGDClassifier
Training took 6.199600696563721
Training RandomForestClassifier
Training took 6.365871429443359
Training AdaBoostClassifier
Training took 122.10607647895813
Training GradientBoostingClassifier
Training took 68.9199686050415
Training BaggingClassifier
Training took 4.300323009490967
Training LinearSVC
Training took 102.1246771812439
Training MLPClassifier
Training took 34.11379957199097
Training KNeighborsClassifier
Training took 0.021413803100585938
Training LinearDiscriminantAnalysis
Training took 1.437546730041504
Training QuadraticDiscriminantAnalysis
Training took 0.7186307907104492


In [198]:
param = {'max_depth':3, 'eta':1, 'objective':'binary:logistic' }
num_round = 20

In [199]:
dtrain = xgb.DMatrix(X_train_1a, label=y_train)
dval = xgb.DMatrix(X_val_1a, label=y_val)

In [200]:
bst = xgb.train(param, dtrain, num_round)

In [201]:
dump(bst,'../glaucoma_project/Models/XGB_clinical_1a_model.joblib')

['../glaucoma_project/Models/XGB_clinical_1a_model.joblib']

In [202]:
y_pred = bst.predict(dtrain)

In [203]:
xgb_train_stats = evaluate_model(y_train, y_pred)

AUC 0.7253054984719369
Youden's J threshold 0.013069622
Accuracy 0.6139156306237723
Sensitivity 0.7225162866449512
Specificity 0.612567628009936


In [204]:
y_pred = bst.predict(dval)

In [205]:
xgb_val_stats = evaluate_model(y_val, y_pred)

AUC 0.7037235782337553
Youden's J threshold 0.012911822
Accuracy 0.6074445143319256
Sensitivity 0.7081911262798635
Specificity 0.6061006146141589


In [206]:
train_stats = {}
val_stats = {}
for model, model_name in zip(models, model_names):
    print(model_name)
    if model_name != 'KNeighborsClassifier':
        print('Training:')
        try:
            y_train_pred_proba = model.predict_proba(X_train_1a)
        except:
            y_train_pred_proba = model._predict_proba_lr(X_train_1a)
        train_stats[model_name] = evaluate_model(y_train,y_train_pred_proba)
    print('Validation:')
    try:
        y_val_pred_proba = model.predict_proba(X_val_1a)
    except:
        y_val_pred_proba = model._predict_proba_lr(X_val_1a)
    val_stats[model_name] = evaluate_model(y_val,y_val_pred_proba)
    print('==========')

LogisticRegression
Training:
AUC 0.7057285592106348
Youden's J threshold 0.49485715134414116
Accuracy 0.583853954767711
Sensitivity 0.7284201954397395
Specificity 0.5820595303375272
Validation:
AUC 0.7162479246769411
Youden's J threshold 0.5220853036884141
Accuracy 0.6292568963968012
Sensitivity 0.689419795221843
Specificity 0.6284543592078307
SGDClassifier
Training:
AUC 0.649151626567193
Youden's J threshold 0.5327124883801883
Accuracy 0.5804469315575213
Sensitivity 0.6433224755700325
Specificity 0.5796664906211543
Validation:
AUC 0.6526442742837077
Youden's J threshold 0.5333056574849587
Accuracy 0.5764893521430496
Sensitivity 0.6621160409556314
Specificity 0.5753471431823355
RandomForestClassifier
Training:
AUC 0.8020774296126552
Youden's J threshold 0.5135665446431534
Accuracy 0.6923295801998288
Sensitivity 0.7502035830618893
Specificity 0.6916112207534917
Validation:
AUC 0.6984736032891297
Youden's J threshold 0.44311456740333555
Accuracy 0.547915356276395
Sensitivity 0.7542662116

In [217]:
df_model_stats = pd.DataFrame()

In [218]:
df_model_stats['Model Names'] = model_names

In [219]:
df_model_stats['Train AUC'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][0] if x in train_stats.keys() else np.nan)
df_model_stats['Train Threshold'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][1] if x in train_stats.keys() else np.nan)
df_model_stats['Train Accuracy'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][2] if x in train_stats.keys() else np.nan)
df_model_stats['Train Sensitivity'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][3] if x in train_stats.keys() else np.nan)
df_model_stats['Train Specificity'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][4] if x in train_stats.keys() else np.nan)

In [220]:
df_model_stats['Val AUC'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][0] if x in val_stats.keys() else np.nan)
df_model_stats['Val Threshold'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][1] if x in val_stats.keys() else np.nan)
df_model_stats['Val Accuracy'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][2] if x in val_stats.keys() else np.nan)
df_model_stats['Val Sensitivity'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][3] if x in val_stats.keys() else np.nan)
df_model_stats['Val Specificity'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][4] if x in val_stats.keys() else np.nan)

In [221]:
df_model_stats = df_model_stats.append({stat_name: stat for stat_name, stat in zip(df_model_stats.columns, ('LogisticRegression (Age Only)',) + age_train_stats + age_val_stats)}, ignore_index=True)

In [222]:
df_model_stats = df_model_stats.append({stat_name: stat for stat_name, stat in zip(df_model_stats.columns, ('XGBoost',) + xgb_train_stats + xgb_val_stats)}, ignore_index=True)

In [223]:
df_model_stats.sort_values('Val AUC', ascending=False).to_csv('../glaucoma_project/UKBB_Data/processed_data/initial_model_selection_clinical_data_1a.csv', index=False)

# Model 1a

In [158]:
models = [LogisticRegression(class_weight='balanced',n_jobs=-1, max_iter=300),
SGDClassifier(class_weight='balanced',n_jobs=-1, random_state=42,loss='modified_huber'),
RandomForestClassifier(class_weight='balanced',max_depth=10, random_state=42, n_jobs=-1),
AdaBoostClassifier(DecisionTreeClassifier(class_weight='balanced',max_depth=10,random_state=42)),
GradientBoostingClassifier(random_state=42),
BaggingClassifier(DecisionTreeClassifier(class_weight='balanced',max_depth=10,random_state=42),n_jobs=-1,random_state=42),
LinearSVC(class_weight='balanced',random_state=42),
MLPClassifier(random_state=42),
KNeighborsClassifier(n_jobs=-1),
LinearDiscriminantAnalysis(),
QuadraticDiscriminantAnalysis()]

In [159]:
model_names = ['LogisticRegression',
               'SGDClassifier',
               'RandomForestClassifier',
               'AdaBoostClassifier',
               'GradientBoostingClassifier',
               'BaggingClassifier',
               'LinearSVC',
               'MLPClassifier',
               'KNeighborsClassifier',
               'LinearDiscriminantAnalysis',
               'QuadraticDiscriminantAnalysis'
]

In [197]:
X_train_1a = X_train[[col for col in classification_dict.keys() if classification_dict[col][1] == '1a']]
X_val_1a = X_val[[col for col in classification_dict.keys() if classification_dict[col][1] == '1a']]

In [174]:
for model, model_name in zip(models, model_names):
    print('Training',model_name)
    t0 = time.time()
    model.fit(X_train_1a,y_train)
    print('Training took',time.time()-t0)
    dump(model,'../glaucoma_project/Models/'+model_name+'_clinical_1a_model.joblib')

Training LogisticRegression
Training took 11.503736019134521
Training SGDClassifier
Training took 6.199600696563721
Training RandomForestClassifier
Training took 6.365871429443359
Training AdaBoostClassifier
Training took 122.10607647895813
Training GradientBoostingClassifier
Training took 68.9199686050415
Training BaggingClassifier
Training took 4.300323009490967
Training LinearSVC
Training took 102.1246771812439
Training MLPClassifier
Training took 34.11379957199097
Training KNeighborsClassifier
Training took 0.021413803100585938
Training LinearDiscriminantAnalysis
Training took 1.437546730041504
Training QuadraticDiscriminantAnalysis
Training took 0.7186307907104492


In [198]:
param = {'max_depth':3, 'eta':1, 'objective':'binary:logistic' }
num_round = 20

In [199]:
dtrain = xgb.DMatrix(X_train_1a, label=y_train)
dval = xgb.DMatrix(X_val_1a, label=y_val)

In [200]:
bst = xgb.train(param, dtrain, num_round)

In [201]:
dump(bst,'../glaucoma_project/Models/XGB_clinical_1a_model.joblib')

['../glaucoma_project/Models/XGB_clinical_1a_model.joblib']

In [202]:
y_pred = bst.predict(dtrain)

In [203]:
xgb_train_stats = evaluate_model(y_train, y_pred)

AUC 0.7253054984719369
Youden's J threshold 0.013069622
Accuracy 0.6139156306237723
Sensitivity 0.7225162866449512
Specificity 0.612567628009936


In [204]:
y_pred = bst.predict(dval)

In [205]:
xgb_val_stats = evaluate_model(y_val, y_pred)

AUC 0.7037235782337553
Youden's J threshold 0.012911822
Accuracy 0.6074445143319256
Sensitivity 0.7081911262798635
Specificity 0.6061006146141589


In [206]:
train_stats = {}
val_stats = {}
for model, model_name in zip(models, model_names):
    print(model_name)
    if model_name != 'KNeighborsClassifier':
        print('Training:')
        try:
            y_train_pred_proba = model.predict_proba(X_train_1a)
        except:
            y_train_pred_proba = model._predict_proba_lr(X_train_1a)
        train_stats[model_name] = evaluate_model(y_train,y_train_pred_proba)
    print('Validation:')
    try:
        y_val_pred_proba = model.predict_proba(X_val_1a)
    except:
        y_val_pred_proba = model._predict_proba_lr(X_val_1a)
    val_stats[model_name] = evaluate_model(y_val,y_val_pred_proba)
    print('==========')

LogisticRegression
Training:
AUC 0.7057285592106348
Youden's J threshold 0.49485715134414116
Accuracy 0.583853954767711
Sensitivity 0.7284201954397395
Specificity 0.5820595303375272
Validation:
AUC 0.7162479246769411
Youden's J threshold 0.5220853036884141
Accuracy 0.6292568963968012
Sensitivity 0.689419795221843
Specificity 0.6284543592078307
SGDClassifier
Training:
AUC 0.649151626567193
Youden's J threshold 0.5327124883801883
Accuracy 0.5804469315575213
Sensitivity 0.6433224755700325
Specificity 0.5796664906211543
Validation:
AUC 0.6526442742837077
Youden's J threshold 0.5333056574849587
Accuracy 0.5764893521430496
Sensitivity 0.6621160409556314
Specificity 0.5753471431823355
RandomForestClassifier
Training:
AUC 0.8020774296126552
Youden's J threshold 0.5135665446431534
Accuracy 0.6923295801998288
Sensitivity 0.7502035830618893
Specificity 0.6916112207534917
Validation:
AUC 0.6984736032891297
Youden's J threshold 0.44311456740333555
Accuracy 0.547915356276395
Sensitivity 0.7542662116

In [217]:
df_model_stats = pd.DataFrame()
df_model_stats['Model Names'] = model_names
df_model_stats['Train AUC'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][0] if x in train_stats.keys() else np.nan)
df_model_stats['Train Threshold'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][1] if x in train_stats.keys() else np.nan)
df_model_stats['Train Accuracy'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][2] if x in train_stats.keys() else np.nan)
df_model_stats['Train Sensitivity'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][3] if x in train_stats.keys() else np.nan)
df_model_stats['Train Specificity'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][4] if x in train_stats.keys() else np.nan)
df_model_stats['Val AUC'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][0] if x in val_stats.keys() else np.nan)
df_model_stats['Val Threshold'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][1] if x in val_stats.keys() else np.nan)
df_model_stats['Val Accuracy'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][2] if x in val_stats.keys() else np.nan)
df_model_stats['Val Sensitivity'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][3] if x in val_stats.keys() else np.nan)
df_model_stats['Val Specificity'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][4] if x in val_stats.keys() else np.nan)

In [221]:
df_model_stats = df_model_stats.append({stat_name: stat for stat_name, stat in zip(df_model_stats.columns, ('LogisticRegression (Age Only)',) + age_train_stats + age_val_stats)}, ignore_index=True)

In [222]:
df_model_stats = df_model_stats.append({stat_name: stat for stat_name, stat in zip(df_model_stats.columns, ('XGBoost',) + xgb_train_stats + xgb_val_stats)}, ignore_index=True)

In [223]:
df_model_stats.sort_values('Val AUC', ascending=False).to_csv('../glaucoma_project/UKBB_Data/processed_data/initial_model_selection_clinical_data_1a.csv', index=False)

Retry models that overfit

In [290]:
df_model_stats = pd.read_csv('../glaucoma_project/UKBB_Data/processed_data/initial_model_selection_clinical_data_1a.csv')

In [292]:
df_model_stats['Train AUC - Val AUC'] = df_model_stats['Train AUC'] - df_model_stats['Val AUC']

In [295]:
df_model_stats[df_model_stats['Train AUC - Val AUC'] > 0.05]

,Model Names,Train AUC,Train Threshold,Train Accuracy,Train Sensitivity,Train Specificity,Val AUC,Val Threshold,Val Accuracy,Val Sensitivity,Val Specificity,Train AUC - Val AUC
7,RandomForestClassifier,0.802077,0.513567,0.692330,0.750204,0.691611,0.698474,0.443115,0.547915,0.754266,0.545163,0.103604
8,BaggingClassifier,0.801756,0.466843,0.654967,0.812704,0.653009,0.689556,0.366118,0.521767,0.767918,0.518484,0.112200
11,AdaBoostClassifier,0.999755,0.501910,0.989437,0.998371,0.989326,0.562313,0.174197,0.543557,0.552901,0.543433,0.437442


In [305]:
models = [
RandomForestClassifier(class_weight='balanced',max_depth=5, random_state=42, n_jobs=-1),
AdaBoostClassifier(DecisionTreeClassifier(class_weight='balanced',max_depth=1,random_state=42)),
BaggingClassifier(DecisionTreeClassifier(class_weight='balanced',max_depth=5,random_state=42),n_jobs=-1,random_state=42),
]

In [306]:
model_names = [
               'RandomForestClassifier',
               'AdaBoostClassifier',
               'BaggingClassifier',
]

In [307]:
for model, model_name in zip(models, model_names):
    print('Training',model_name)
    t0 = time.time()
    model.fit(X_train_1a,y_train)
    print('Training took',time.time()-t0)
    dump(model,'../glaucoma_project/Models/'+model_name+'_clinical_1a_model_v2.joblib')

Training RandomForestClassifier
Training took 3.575226068496704
Training AdaBoostClassifier
Training took 17.460238456726074
Training BaggingClassifier
Training took 2.148622989654541


In [308]:
train_stats = {}
val_stats = {}
for model, model_name in zip(models, model_names):
    print(model_name)
    if model_name != 'KNeighborsClassifier':
        print('Training:')
        try:
            y_train_pred_proba = model.predict_proba(X_train_1a)
        except:
            y_train_pred_proba = model._predict_proba_lr(X_train_1a)
        train_stats[model_name] = evaluate_model(y_train,y_train_pred_proba)
    print('Validation:')
    try:
        y_val_pred_proba = model.predict_proba(X_val_1a)
    except:
        y_val_pred_proba = model._predict_proba_lr(X_val_1a)
    val_stats[model_name] = evaluate_model(y_val,y_val_pred_proba)
    print('==========')

RandomForestClassifier
Training:
AUC 0.7128004519418345
Youden's J threshold 0.5258106994061778
Accuracy 0.5852292439902856
Sensitivity 0.7345276872964169
Specificity 0.583376081226894
Validation:
AUC 0.7040504440433859
Youden's J threshold 0.513072842170541
Accuracy 0.5457363644532303
Sensitivity 0.7679180887372014
Specificity 0.542772592761211
AdaBoostClassifier
Training:
AUC 0.7081311509421434
Youden's J threshold 0.4994751441959101
Accuracy 0.5515733458465517
Sensitivity 0.7624185667752443
Specificity 0.5489562354225471
Validation:
AUC 0.7134981070567588
Youden's J threshold 0.5001267535422818
Accuracy 0.6058945098391589
Sensitivity 0.7150170648464164
Specificity 0.6044388800364215
BaggingClassifier
Training:
AUC 0.7152395074975955
Youden's J threshold 0.4928117370399569
Accuracy 0.5722875477669646
Sensitivity 0.7534609120521173
Specificity 0.5700387384359578
Validation:
AUC 0.7035077912502747
Youden's J threshold 0.4582739650341653
Accuracy 0.4905876538772576
Sensitivity 0.8242320

In [310]:
df_model_stats = pd.DataFrame()
df_model_stats['Model Names'] = model_names
df_model_stats['Train AUC'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][0] if x in train_stats.keys() else np.nan)
df_model_stats['Train Threshold'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][1] if x in train_stats.keys() else np.nan)
df_model_stats['Train Accuracy'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][2] if x in train_stats.keys() else np.nan)
df_model_stats['Train Sensitivity'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][3] if x in train_stats.keys() else np.nan)
df_model_stats['Train Specificity'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][4] if x in train_stats.keys() else np.nan)
df_model_stats['Val AUC'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][0] if x in val_stats.keys() else np.nan)
df_model_stats['Val Threshold'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][1] if x in val_stats.keys() else np.nan)
df_model_stats['Val Accuracy'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][2] if x in val_stats.keys() else np.nan)
df_model_stats['Val Sensitivity'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][3] if x in val_stats.keys() else np.nan)
df_model_stats['Val Specificity'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][4] if x in val_stats.keys() else np.nan)

In [312]:
df_model_stats.sort_values('Val AUC', ascending=False).to_csv('../glaucoma_project/UKBB_Data/processed_data/initial_model_selection_clinical_data_1a_supplemental.csv', index=False)

Best model is logistic regression...

# Model 1b

In [314]:
models = [LogisticRegression(class_weight='balanced',n_jobs=-1, max_iter=300),
SGDClassifier(class_weight='balanced',n_jobs=-1, random_state=42,loss='modified_huber'),
RandomForestClassifier(class_weight='balanced',max_depth=10, random_state=42, n_jobs=-1),
AdaBoostClassifier(DecisionTreeClassifier(class_weight='balanced',max_depth=10,random_state=42)),
GradientBoostingClassifier(random_state=42),
BaggingClassifier(DecisionTreeClassifier(class_weight='balanced',max_depth=10,random_state=42),n_jobs=-1,random_state=42),
LinearSVC(class_weight='balanced',random_state=42),
MLPClassifier(random_state=42),
KNeighborsClassifier(n_jobs=-1),
LinearDiscriminantAnalysis(),
QuadraticDiscriminantAnalysis()]

In [225]:
model_names = ['LogisticRegression',
               'SGDClassifier',
               'RandomForestClassifier',
               'AdaBoostClassifier',
               'GradientBoostingClassifier',
               'BaggingClassifier',
               'LinearSVC',
               'MLPClassifier',
               'KNeighborsClassifier',
               'LinearDiscriminantAnalysis',
               'QuadraticDiscriminantAnalysis'
]

In [247]:
X_train_1b = X_train[[col for col in classification_dict.keys() if classification_dict[col][1] in ['1a','1b']]]
X_val_1b = X_val[[col for col in classification_dict.keys() if classification_dict[col][1] in ['1a','1b']]]

In [232]:
for model, model_name in zip(models, model_names):
    print('Training',model_name)
    t0 = time.time()
    model.fit(X_train_1b,y_train)
    print('Training took',time.time()-t0)
    dump(model,'../glaucoma_project/Models/'+model_name+'_clinical_1b_model.joblib')

Training LogisticRegression


/home/cmay091/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training took 236.87725973129272
Training SGDClassifier
Training took 12.96200442314148
Training RandomForestClassifier
Training took 9.397969007492065
Training AdaBoostClassifier
Training took 402.98415350914
Training GradientBoostingClassifier
Training took 209.67638063430786
Training BaggingClassifier
Training took 14.783742189407349
Training LinearSVC
Training took 218.02727842330933
Training MLPClassifier
Training took 493.29800844192505
Training KNeighborsClassifier
Training took 0.04703474044799805
Training LinearDiscriminantAnalysis
Training took 8.286629915237427
Training QuadraticDiscriminantAnalysis
Training took 4.433009386062622


In [233]:
param = {'max_depth':3, 'eta':1, 'objective':'binary:logistic' }
num_round = 20

In [249]:
dtrain = xgb.DMatrix(X_train_1b, label=y_train)
dval = xgb.DMatrix(X_val_1b, label=y_val)

In [251]:
bst = xgb.train(param, dtrain, num_round)

In [252]:
dump(bst,'../glaucoma_project/Models/XGB_clinical_1b_model.joblib')

['../glaucoma_project/Models/XGB_clinical_1b_model.joblib']

In [253]:
y_pred = bst.predict(dtrain)

In [254]:
xgb_train_stats = evaluate_model(y_train, y_pred)

AUC 0.7359755150524758
Youden's J threshold 0.01030101
Accuracy 0.5592609879618513
Sensitivity 0.7884771986970684
Specificity 0.5564158481392664


In [255]:
y_pred = bst.predict(dval)

In [256]:
xgb_val_stats = evaluate_model(y_val, y_pred)

AUC 0.7041367394140072
Youden's J threshold 0.009612103
Accuracy 0.5306855961901339
Sensitivity 0.7815699658703071
Specificity 0.5273389483268837


In [257]:
train_stats = {}
val_stats = {}
for model, model_name in zip(models, model_names):
    print(model_name)
    if model_name != 'KNeighborsClassifier':
        print('Training:')
        try:
            y_train_pred_proba = model.predict_proba(X_train_1b)
        except:
            y_train_pred_proba = model._predict_proba_lr(X_train_1b)
        train_stats[model_name] = evaluate_model(y_train,y_train_pred_proba)
    print('Validation:')
    try:
        y_val_pred_proba = model.predict_proba(X_val_1b)
    except:
        y_val_pred_proba = model._predict_proba_lr(X_val_1b)
    val_stats[model_name] = evaluate_model(y_val,y_val_pred_proba)
    print('==========')

LogisticRegression
Training:
AUC 0.7149258240534646
Youden's J threshold 0.5047124273227159
Accuracy 0.613436400985416
Sensitivity 0.7125407166123778
Specificity 0.6122062714318565
Validation:
AUC 0.7154512414646634
Youden's J threshold 0.432406724444531
Accuracy 0.49660796118249617
Sensitivity 0.8344709897610921
Specificity 0.49210106988390623
SGDClassifier
Training:
AUC 0.6329725485986157
Youden's J threshold 0.29044532317489635
Accuracy 0.44447300963700853
Sensitivity 0.7567182410423453
Specificity 0.44059727441115304
Validation:
AUC 0.629258112308676
Youden's J threshold 0.3160180498434062
Accuracy 0.4668883098211879
Sensitivity 0.7337883959044369
Specificity 0.46332802185294786
RandomForestClassifier
Training:
AUC 0.8425979534085672
Youden's J threshold 0.5187069356267482
Accuracy 0.7647232074440338
Sensitivity 0.7477605863192183
Specificity 0.7649337555056339
Validation:
AUC 0.687197791397888
Youden's J threshold 0.43603520352576325
Accuracy 0.5393341719831072
Sensitivity 0.76109

In [258]:
df_model_stats = pd.DataFrame()

In [259]:
df_model_stats['Model Names'] = model_names

In [260]:
df_model_stats['Train AUC'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][0] if x in train_stats.keys() else np.nan)
df_model_stats['Train Threshold'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][1] if x in train_stats.keys() else np.nan)
df_model_stats['Train Accuracy'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][2] if x in train_stats.keys() else np.nan)
df_model_stats['Train Sensitivity'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][3] if x in train_stats.keys() else np.nan)
df_model_stats['Train Specificity'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][4] if x in train_stats.keys() else np.nan)

In [261]:
df_model_stats['Val AUC'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][0] if x in val_stats.keys() else np.nan)
df_model_stats['Val Threshold'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][1] if x in val_stats.keys() else np.nan)
df_model_stats['Val Accuracy'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][2] if x in val_stats.keys() else np.nan)
df_model_stats['Val Sensitivity'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][3] if x in val_stats.keys() else np.nan)
df_model_stats['Val Specificity'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][4] if x in val_stats.keys() else np.nan)

In [262]:
df_model_stats = df_model_stats.append({stat_name: stat for stat_name, stat in zip(df_model_stats.columns, ('LogisticRegression (Age Only)',) + age_train_stats + age_val_stats)}, ignore_index=True)

In [263]:
df_model_stats = df_model_stats.append({stat_name: stat for stat_name, stat in zip(df_model_stats.columns, ('XGBoost',) + xgb_train_stats + xgb_val_stats)}, ignore_index=True)

In [264]:
df_model_stats.sort_values('Val AUC', ascending=False).to_csv('../glaucoma_project/UKBB_Data/processed_data/initial_model_selection_clinical_data_1b.csv', index=False)

Retry models that overfit

In [315]:
df_model_stats = pd.read_csv('../glaucoma_project/UKBB_Data/processed_data/initial_model_selection_clinical_data_1b.csv')

In [316]:
df_model_stats['Train AUC - Val AUC'] = df_model_stats['Train AUC'] - df_model_stats['Val AUC']

In [317]:
df_model_stats[df_model_stats['Train AUC - Val AUC'] > 0.05]

,Model Names,Train AUC,Train Threshold,Train Accuracy,Train Sensitivity,Train Specificity,Val AUC,Val Threshold,Val Accuracy,Val Sensitivity,Val Specificity,Train AUC - Val AUC
6,RandomForestClassifier,0.842598,0.518707,0.764723,0.747761,0.764934,0.687198,0.436035,0.539334,0.761092,0.536376,0.155400
7,BaggingClassifier,0.833062,0.489170,0.732283,0.783795,0.731643,0.684273,0.447936,0.661515,0.612628,0.662167,0.148789
9,MLPClassifier,0.918669,0.011179,0.818075,0.846702,0.817720,0.629307,0.001981,0.527383,0.679181,0.525359,0.289362
12,AdaBoostClassifier,1.000000,0.530496,0.999998,0.999796,1.000000,0.502564,0.037437,0.373753,0.646758,0.370112,0.497436


In [327]:
models = [ RandomForestClassifier(class_weight='balanced',max_depth=5, random_state=42, n_jobs=-1),
AdaBoostClassifier(DecisionTreeClassifier(class_weight='balanced',max_depth=1,random_state=42)),
BaggingClassifier(DecisionTreeClassifier(class_weight='balanced',max_depth=5,random_state=42),n_jobs=-1,random_state=42),
MLPClassifier(random_state=42,alpha=0.1)]

In [328]:
model_names = ['RandomForestClassifier',
               'AdaBoostClassifier',
               'BaggingClassifier',
               'MLPClassifier',
]

In [329]:
for model, model_name in zip(models, model_names):
    print('Training',model_name)
    t0 = time.time()
    model.fit(X_train_1b,y_train)
    print('Training took',time.time()-t0)
    dump(model,'../glaucoma_project/Models/'+model_name+'_clinical_1b_model_v2.joblib')

Training RandomForestClassifier
Training took 5.028035879135132
Training AdaBoostClassifier
Training took 50.71330285072327
Training BaggingClassifier
Training took 7.018242835998535
Training MLPClassifier
Training took 70.10692048072815


In [330]:
train_stats = {}
val_stats = {}
for model, model_name in zip(models, model_names):
    print(model_name)
    if model_name != 'KNeighborsClassifier':
        print('Training:')
        try:
            y_train_pred_proba = model.predict_proba(X_train_1b)
        except:
            y_train_pred_proba = model._predict_proba_lr(X_train_1b)
        train_stats[model_name] = evaluate_model(y_train,y_train_pred_proba)
    print('Validation:')
    try:
        y_val_pred_proba = model.predict_proba(X_val_1b)
    except:
        y_val_pred_proba = model._predict_proba_lr(X_val_1b)
    val_stats[model_name] = evaluate_model(y_val,y_val_pred_proba)
    print('==========')

RandomForestClassifier
Training:
AUC 0.7116098256011056
Youden's J threshold 0.5070441404444831
Accuracy 0.5740122752675075
Sensitivity 0.7463355048859935
Specificity 0.5718733179862078
Validation:
AUC 0.6993571451323817
Youden's J threshold 0.4968976434809643
Accuracy 0.5357848863330039
Sensitivity 0.7679180887372014
Specificity 0.5326883678579558
AdaBoostClassifier
Training:
AUC 0.7131828476467212
Youden's J threshold 0.4995672125601444
Accuracy 0.5672132047733269
Sensitivity 0.7497964169381107
Specificity 0.5649468957448368
Validation:
AUC 0.7055707420042279
Youden's J threshold 0.49890898725813754
Accuracy 0.5152529427621529
Sensitivity 0.8088737201365188
Specificity 0.5113362167083997
BaggingClassifier
Training:
AUC 0.7205130115880793
Youden's J threshold 0.4997281273510715
Accuracy 0.579373656846619
Sensitivity 0.7514250814332247
Specificity 0.5772380733376966
Validation:
AUC 0.7033803001828071
Youden's J threshold 0.5060793731917362
Accuracy 0.5947973762242789
Sensitivity 0.7184

In [331]:
df_model_stats = pd.DataFrame()
df_model_stats['Model Names'] = model_names
df_model_stats['Train AUC'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][0] if x in train_stats.keys() else np.nan)
df_model_stats['Train Threshold'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][1] if x in train_stats.keys() else np.nan)
df_model_stats['Train Accuracy'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][2] if x in train_stats.keys() else np.nan)
df_model_stats['Train Sensitivity'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][3] if x in train_stats.keys() else np.nan)
df_model_stats['Train Specificity'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][4] if x in train_stats.keys() else np.nan)
df_model_stats['Val AUC'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][0] if x in val_stats.keys() else np.nan)
df_model_stats['Val Threshold'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][1] if x in val_stats.keys() else np.nan)
df_model_stats['Val Accuracy'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][2] if x in val_stats.keys() else np.nan)
df_model_stats['Val Sensitivity'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][3] if x in val_stats.keys() else np.nan)
df_model_stats['Val Specificity'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][4] if x in val_stats.keys() else np.nan)

In [333]:
df_model_stats.sort_values('Val AUC', ascending=False).to_csv('../glaucoma_project/UKBB_Data/processed_data/initial_model_selection_clinical_data_1b_supplemental.csv', index=False)

Best model was GradientBoostingClassifier...

# Model 2

In [265]:
models = [LogisticRegression(class_weight='balanced',n_jobs=-1, max_iter=300),
SGDClassifier(class_weight='balanced',n_jobs=-1, random_state=42,loss='modified_huber'),
RandomForestClassifier(class_weight='balanced',max_depth=10, random_state=42, n_jobs=-1),
AdaBoostClassifier(DecisionTreeClassifier(class_weight='balanced',max_depth=10,random_state=42)),
GradientBoostingClassifier(random_state=42),
BaggingClassifier(DecisionTreeClassifier(class_weight='balanced',max_depth=10,random_state=42),n_jobs=-1,random_state=42),
LinearSVC(class_weight='balanced',random_state=42),
MLPClassifier(random_state=42),
KNeighborsClassifier(n_jobs=-1),
LinearDiscriminantAnalysis(),
QuadraticDiscriminantAnalysis()]

In [266]:
model_names = ['LogisticRegression',
               'SGDClassifier',
               'RandomForestClassifier',
               'AdaBoostClassifier',
               'GradientBoostingClassifier',
               'BaggingClassifier',
               'LinearSVC',
               'MLPClassifier',
               'KNeighborsClassifier',
               'LinearDiscriminantAnalysis',
               'QuadraticDiscriminantAnalysis'
]

In [267]:
X_train_2 = X_train[[col for col in classification_dict.keys() if classification_dict[col][1] in ['1a','1b','2']]]
X_val_2 = X_val[[col for col in classification_dict.keys() if classification_dict[col][1] in ['1a','1b','2']]]

In [272]:
for model, model_name in zip(models, model_names):
    print('Training',model_name)
    t0 = time.time()
    model.fit(X_train_2,y_train)
    print('Training took',time.time()-t0)
    dump(model,'../glaucoma_project/Models/'+model_name+'_clinical_2_model.joblib')

Training LogisticRegression


/home/cmay091/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training took 259.79570508003235
Training SGDClassifier
Training took 13.874680519104004
Training RandomForestClassifier
Training took 9.305098295211792
Training AdaBoostClassifier
Training took 430.17840456962585
Training GradientBoostingClassifier
Training took 227.20680451393127
Training BaggingClassifier
Training took 15.560014486312866
Training LinearSVC
Training took 233.28614401817322
Training MLPClassifier
Training took 362.91291403770447
Training KNeighborsClassifier
Training took 0.04902935028076172
Training LinearDiscriminantAnalysis
Training took 8.932491540908813
Training QuadraticDiscriminantAnalysis
Training took 4.735562086105347


In [273]:
param = {'max_depth':3, 'eta':1, 'objective':'binary:logistic' }
num_round = 20

In [274]:
dtrain = xgb.DMatrix(X_train_2, label=y_train)
dval = xgb.DMatrix(X_val_2, label=y_val)

In [275]:
bst = xgb.train(param, dtrain, num_round)

In [276]:
dump(bst,'../glaucoma_project/Models/XGB_clinical_2_model.joblib')

['../glaucoma_project/Models/XGB_clinical_2_model.joblib']

In [277]:
y_pred = bst.predict(dtrain)

In [278]:
xgb_train_stats = evaluate_model(y_train, y_pred)

AUC 0.7595164716590088
Youden's J threshold 0.0111725405
Accuracy 0.6041762866192596
Sensitivity 0.7785016286644951
Specificity 0.6020124781733046


In [279]:
y_pred = bst.predict(dval)

In [280]:
xgb_val_stats = evaluate_model(y_val, y_pred)

AUC 0.7271068267931685
Youden's J threshold 0.0094262175
Accuracy 0.5411088148081589
Sensitivity 0.8020477815699659
Specificity 0.5376280446164352


In [281]:
train_stats = {}
val_stats = {}
for model, model_name in zip(models, model_names):
    print(model_name)
    if model_name != 'KNeighborsClassifier':
        print('Training:')
        try:
            y_train_pred_proba = model.predict_proba(X_train_2)
        except:
            y_train_pred_proba = model._predict_proba_lr(X_train_2)
        train_stats[model_name] = evaluate_model(y_train,y_train_pred_proba)
    print('Validation:')
    try:
        y_val_pred_proba = model.predict_proba(X_val_2)
    except:
        y_val_pred_proba = model._predict_proba_lr(X_val_2)
    val_stats[model_name] = evaluate_model(y_val,y_val_pred_proba)
    print('==========')

LogisticRegression
Training:
AUC 0.7331040391156046
Youden's J threshold 0.4771157881321405
Accuracy 0.5958921034437142
Sensitivity 0.755700325732899
Specificity 0.5939084883418282
Validation:
AUC 0.7304280623300022
Youden's J threshold 0.44794678911480407
Accuracy 0.547555934944739
Sensitivity 0.8037542662116041
Specificity 0.5441384020031869
SGDClassifier
Training:
AUC 0.6447419140220254
Youden's J threshold 0.08969045424608396
Accuracy 0.5637887096492389
Sensitivity 0.6703990228013029
Specificity 0.562465412110752
Validation:
AUC 0.6486540796753134
Youden's J threshold 0.07105748237657461
Accuracy 0.542546500134783
Sensitivity 0.7167235494880546
Specificity 0.5402230821761894
RandomForestClassifier
Training:
AUC 0.8299613056522446
Youden's J threshold 0.5355903535949916
Accuracy 0.7535336945859531
Sensitivity 0.7253664495114006
Specificity 0.7538833197298165
Validation:
AUC 0.7106156319120787
Youden's J threshold 0.4331557101493894
Accuracy 0.5590349537245035
Sensitivity 0.771331058

In [282]:
df_model_stats = pd.DataFrame()

In [283]:
df_model_stats['Model Names'] = model_names

In [284]:
df_model_stats['Train AUC'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][0] if x in train_stats.keys() else np.nan)
df_model_stats['Train Threshold'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][1] if x in train_stats.keys() else np.nan)
df_model_stats['Train Accuracy'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][2] if x in train_stats.keys() else np.nan)
df_model_stats['Train Sensitivity'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][3] if x in train_stats.keys() else np.nan)
df_model_stats['Train Specificity'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][4] if x in train_stats.keys() else np.nan)

In [285]:
df_model_stats['Val AUC'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][0] if x in val_stats.keys() else np.nan)
df_model_stats['Val Threshold'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][1] if x in val_stats.keys() else np.nan)
df_model_stats['Val Accuracy'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][2] if x in val_stats.keys() else np.nan)
df_model_stats['Val Sensitivity'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][3] if x in val_stats.keys() else np.nan)
df_model_stats['Val Specificity'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][4] if x in val_stats.keys() else np.nan)

In [286]:
df_model_stats = df_model_stats.append({stat_name: stat for stat_name, stat in zip(df_model_stats.columns, ('LogisticRegression (Age Only)',) + age_train_stats + age_val_stats)}, ignore_index=True)

In [287]:
df_model_stats = df_model_stats.append({stat_name: stat for stat_name, stat in zip(df_model_stats.columns, ('XGBoost',) + xgb_train_stats + xgb_val_stats)}, ignore_index=True)

In [288]:
df_model_stats.sort_values('Val AUC', ascending=False).to_csv('../glaucoma_project/UKBB_Data/processed_data/initial_model_selection_clinical_data_2.csv', index=False)

Retry models that overfit

In [349]:
df_model_stats = pd.read_csv('../glaucoma_project/UKBB_Data/processed_data/initial_model_selection_clinical_data_2.csv')

In [350]:
df_model_stats['Train AUC - Val AUC'] = df_model_stats['Train AUC'] - df_model_stats['Val AUC']

In [351]:
df_model_stats[df_model_stats['Train AUC - Val AUC'] > 0.05]

,Model Names,Train AUC,Train Threshold,Train Accuracy,Train Sensitivity,Train Specificity,Val AUC,Val Threshold,Val Accuracy,Val Sensitivity,Val Specificity,Train AUC - Val AUC
4,RandomForestClassifier,0.829961,0.535590,0.753534,0.725366,0.753883,0.710616,0.433156,0.559035,0.771331,0.556203,0.119346
7,BaggingClassifier,0.813853,0.437254,0.624454,0.862581,0.621498,0.703816,0.455669,0.641904,0.677474,0.641430,0.110037
10,MLPClassifier,0.934568,0.016336,0.848246,0.857288,0.848134,0.595805,0.001472,0.498225,0.653584,0.496153,0.338763
11,AdaBoostClassifier,1.000000,0.527938,0.999998,0.999796,1.000000,0.531298,0.046462,0.369755,0.687713,0.365513,0.468702


In [353]:
models = [ RandomForestClassifier(class_weight='balanced',max_depth=5, random_state=42, n_jobs=-1),
AdaBoostClassifier(DecisionTreeClassifier(class_weight='balanced',max_depth=1,random_state=42)),
BaggingClassifier(DecisionTreeClassifier(class_weight='balanced',max_depth=5,random_state=42),n_jobs=-1,random_state=42),
MLPClassifier(random_state=42,alpha=0.1)]

In [354]:
model_names = ['RandomForestClassifier',
               'AdaBoostClassifier',
               'BaggingClassifier',
               'MLPClassifier',
]

In [355]:
for model, model_name in zip(models, model_names):
    print('Training',model_name)
    t0 = time.time()
    model.fit(X_train_2,y_train)
    print('Training took',time.time()-t0)
    dump(model,'../glaucoma_project/Models/'+model_name+'_clinical_2_model_v2.joblib')

Training RandomForestClassifier
Training took 4.929687261581421
Training AdaBoostClassifier
Training took 53.6724808216095
Training BaggingClassifier
Training took 8.538477182388306
Training MLPClassifier
Training took 86.29969811439514


In [356]:
train_stats = {}
val_stats = {}
for model, model_name in zip(models, model_names):
    print(model_name)
    if model_name != 'KNeighborsClassifier':
        print('Training:')
        try:
            y_train_pred_proba = model.predict_proba(X_train_2)
        except:
            y_train_pred_proba = model._predict_proba_lr(X_train_2)
        train_stats[model_name] = evaluate_model(y_train,y_train_pred_proba)
    print('Validation:')
    try:
        y_val_pred_proba = model.predict_proba(X_val_2)
    except:
        y_val_pred_proba = model._predict_proba_lr(X_val_2)
    val_stats[model_name] = evaluate_model(y_val,y_val_pred_proba)
    print('==========')

RandomForestClassifier
Training:
AUC 0.735786956190184
Youden's J threshold 0.48788904875515626
Accuracy 0.5515933137481498
Sensitivity 0.8102605863192183
Specificity 0.5483826134419593
Validation:
AUC 0.7208655718957168
Youden's J threshold 0.49730857815933743
Accuracy 0.5678407763500763
Sensitivity 0.7764505119453925
Specificity 0.5650580468927839
AdaBoostClassifier
Training:
AUC 0.7376436534076873
Youden's J threshold 0.4998510520764763
Accuracy 0.6054018165798479
Sensitivity 0.7471498371335505
Specificity 0.6036423732282789
Validation:
AUC 0.7294398123294195
Youden's J threshold 0.49842589610832866
Accuracy 0.5235870248899273
Sensitivity 0.8191126279863481
Specificity 0.5196448895970863
BaggingClassifier
Training:
AUC 0.7352709914950863
Youden's J threshold 0.5127882434402917
Accuracy 0.6254121499689249
Sensitivity 0.7294381107491856
Specificity 0.6241209306321719
Validation:
AUC 0.7147220523808822
Youden's J threshold 0.5066456389152408
Accuracy 0.5983466618743822
Sensitivity 0.72

In [357]:
df_model_stats = pd.DataFrame()
df_model_stats['Model Names'] = model_names
df_model_stats['Train AUC'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][0] if x in train_stats.keys() else np.nan)
df_model_stats['Train Threshold'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][1] if x in train_stats.keys() else np.nan)
df_model_stats['Train Accuracy'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][2] if x in train_stats.keys() else np.nan)
df_model_stats['Train Sensitivity'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][3] if x in train_stats.keys() else np.nan)
df_model_stats['Train Specificity'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][4] if x in train_stats.keys() else np.nan)
df_model_stats['Val AUC'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][0] if x in val_stats.keys() else np.nan)
df_model_stats['Val Threshold'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][1] if x in val_stats.keys() else np.nan)
df_model_stats['Val Accuracy'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][2] if x in val_stats.keys() else np.nan)
df_model_stats['Val Sensitivity'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][3] if x in val_stats.keys() else np.nan)
df_model_stats['Val Specificity'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][4] if x in val_stats.keys() else np.nan)

In [358]:
df_model_stats.sort_values('Val AUC', ascending=False).to_csv('../glaucoma_project/UKBB_Data/processed_data/initial_model_selection_clinical_data_2_supplemental.csv', index=False)

Best model was GradientBoostingClassifier again...

# Model 3

## Pipeline to try the other models

In [65]:
models = [LogisticRegression(class_weight='balanced',n_jobs=-1, max_iter=300),
SGDClassifier(class_weight='balanced',n_jobs=-1, random_state=42,loss='modified_huber'),
RandomForestClassifier(class_weight='balanced',max_depth=10, random_state=42, n_jobs=-1),
AdaBoostClassifier(DecisionTreeClassifier(class_weight='balanced',max_depth=10,random_state=42)),
GradientBoostingClassifier(random_state=42),
BaggingClassifier(DecisionTreeClassifier(class_weight='balanced',max_depth=10,random_state=42),n_jobs=-1,random_state=42),
LinearSVC(class_weight='balanced',random_state=42),
MLPClassifier(random_state=42),
KNeighborsClassifier(n_jobs=-1),
LinearDiscriminantAnalysis(),
QuadraticDiscriminantAnalysis()]

In [66]:
model_names = ['LogisticRegression',
               'SGDClassifier',
               'RandomForestClassifier',
               'AdaBoostClassifier',
               'GradientBoostingClassifier',
               'BaggingClassifier',
               'LinearSVC',
               'MLPClassifier',
               'KNeighborsClassifier',
               'LinearDiscriminantAnalysis',
               'QuadraticDiscriminantAnalysis'
]

In [67]:
for model, model_name in zip(models, model_names):
    print('Training',model_name)
    t0 = time.time()
    model.fit(X_train,y_train)
    print('Training took',time.time()-t0)
    dump(model,'../glaucoma_project/Models/'+model_name+'_clinical_initial_model.joblib')

Training LogisticRegression


/home/cmay091/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training took 582.5166902542114
Training SGDClassifier
Training took 21.71672797203064
Training RandomForestClassifier
Training took 13.527515411376953
Training AdaBoostClassifier
Training took 842.5735867023468
Training GradientBoostingClassifier
Training took 410.1408112049103
Training BaggingClassifier
Training took 31.87856674194336
Training LinearSVC
Training took 300.7420377731323
Training MLPClassifier
Training took 813.8047106266022
Training KNeighborsClassifier
Training took 0.07845449447631836
Training LinearDiscriminantAnalysis
Training took 19.8011736869812
Training QuadraticDiscriminantAnalysis
Training took 10.482309579849243


## Training and testing XGBoost

In [69]:
param = {'max_depth':3, 'eta':1, 'objective':'binary:logistic' }
num_round = 20

In [70]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

In [71]:
bst = xgb.train(param, dtrain, num_round)

In [72]:
dump(bst,'../glaucoma_project/Models/XGB_clinical_initial_model.joblib')

['../glaucoma_project/Models/XGB_clinical_initial_model.joblib']

In [119]:
y_pred = bst.predict(dtrain)

In [120]:
xgb_train_stats = evaluate_model(y_train, y_pred)

AUC 0.7618728026738328
Youden's J threshold 0.012641026
Accuracy 0.666224044847907
Sensitivity 0.7198697068403909
Specificity 0.6655581695646791


In [121]:
y_pred = bst.predict(dval)

In [122]:
xgb_val_stats = evaluate_model(y_val, y_pred)

AUC 0.7357737721118535
Youden's J threshold 0.009801479
Accuracy 0.579140084464013
Sensitivity 0.7832764505119454
Specificity 0.5764170270885499


## Validate models

In [77]:
train_stats = {}
val_stats = {}
for model, model_name in zip(models, model_names):
    print(model_name)
    if model_name != 'KNeighborsClassifier':
        print('Training:')
        try:
            y_train_pred_proba = model.predict_proba(X_train)
        except:
            y_train_pred_proba = model._predict_proba_lr(X_train)
        train_stats[model_name] = evaluate_model(y_train,y_train_pred_proba)
    print('Validation:')
    try:
        y_val_pred_proba = model.predict_proba(X_val)
    except:
        y_val_pred_proba = model._predict_proba_lr(X_val)
    val_stats[model_name] = evaluate_model(y_val,y_val_pred_proba)
    print('==========')

LogisticRegression
Training:
AUC 0.7403546634642258
Youden's J threshold 0.48132248829273966
Accuracy 0.6133789932683211
Sensitivity 0.7449104234527687
Specificity 0.6117463630597553
Validation:
AUC 0.7328964245786619
Youden's J threshold 0.4863427841390447
Accuracy 0.6197546949411448
Sensitivity 0.7286689419795221
Specificity 0.6183018438424767
SGDClassifier
Training:
AUC 0.633805178479044
Youden's J threshold 0.05503207607983851
Accuracy 0.5756571311616576
Sensitivity 0.6502442996742671
Specificity 0.574731320012837
Validation:
AUC 0.6487846006950245
Youden's J threshold 0.11280181466850592
Accuracy 0.6037379818492228
Sensitivity 0.6484641638225256
Specificity 0.6031413612565445
RandomForestClassifier
Training:
AUC 0.8514694892621799
Youden's J threshold 0.5302665303021459
Accuracy 0.789648140613963
Sensitivity 0.7274022801302932
Specificity 0.7904207656210911
Validation:
AUC 0.7112326544945456
Youden's J threshold 0.42739746868157985
Accuracy 0.5449950579566897
Sensitivity 0.7815699

In [125]:
df_model_stats = pd.DataFrame()

In [126]:
df_model_stats['Model Names'] = model_names

In [127]:
df_model_stats['Train AUC'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][0] if x in train_stats.keys() else np.nan)
df_model_stats['Train Threshold'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][1] if x in train_stats.keys() else np.nan)
df_model_stats['Train Accuracy'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][2] if x in train_stats.keys() else np.nan)
df_model_stats['Train Sensitivity'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][3] if x in train_stats.keys() else np.nan)
df_model_stats['Train Specificity'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][4] if x in train_stats.keys() else np.nan)

In [128]:
df_model_stats['Val AUC'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][0] if x in val_stats.keys() else np.nan)
df_model_stats['Val Threshold'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][1] if x in val_stats.keys() else np.nan)
df_model_stats['Val Accuracy'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][2] if x in val_stats.keys() else np.nan)
df_model_stats['Val Sensitivity'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][3] if x in val_stats.keys() else np.nan)
df_model_stats['Val Specificity'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][4] if x in val_stats.keys() else np.nan)

In [132]:
df_model_stats = df_model_stats.append({stat_name: stat for stat_name, stat in zip(df_model_stats.columns, ('LogisticRegression (Age Only)',) + age_train_stats + age_val_stats)}, ignore_index=True)

In [133]:
df_model_stats = df_model_stats.append({stat_name: stat for stat_name, stat in zip(df_model_stats.columns, ('XGBoost',) + xgb_train_stats + xgb_val_stats)}, ignore_index=True)

In [135]:
df_model_stats.sort_values('Val AUC', ascending=False).to_csv('../glaucoma_project/UKBB_Data/processed_data/initial_model_selection_clinical_data_maximal_only.csv', index=False)

## Retry models that overfit

In [360]:
df_model_stats = pd.read_csv('../glaucoma_project/UKBB_Data/processed_data/initial_model_selection_clinical_data_maximal_only.csv')

In [361]:
df_model_stats['Train AUC - Val AUC'] = df_model_stats['Train AUC'] - df_model_stats['Val AUC']

In [362]:
df_model_stats[df_model_stats['Train AUC - Val AUC'] > 0.05]

,Model Names,Train AUC,Train Threshold,Train Accuracy,Train Sensitivity,Train Specificity,Val AUC,Val Threshold,Val Accuracy,Val Sensitivity,Val Specificity,Train AUC - Val AUC
4,RandomForestClassifier,0.851469,0.530267,0.789648,0.727402,0.790421,0.711233,4.273975e-01,0.544995,0.781570,0.541839,0.140237
5,BaggingClassifier,0.822152,0.460424,0.658696,0.839984,0.656446,0.708082,3.253710e-01,0.518488,0.802048,0.514705,0.114071
9,QuadraticDiscriminantAnalysis,0.747448,0.005467,0.653292,0.734324,0.652287,0.632108,1.134979e-04,0.498136,0.728669,0.495060,0.115340
10,MLPClassifier,0.998869,0.051680,0.987325,0.985953,0.987342,0.616323,2.242868e-10,0.532999,0.639932,0.531573,0.382547
11,AdaBoostClassifier,1.000000,0.544689,0.999998,0.999796,1.000000,0.544656,1.087930e-01,0.581454,0.494881,0.582609,0.455344


In [364]:
models = [ RandomForestClassifier(class_weight='balanced',max_depth=5, random_state=42, n_jobs=-1),
AdaBoostClassifier(DecisionTreeClassifier(class_weight='balanced',max_depth=1,random_state=42)),
BaggingClassifier(DecisionTreeClassifier(class_weight='balanced',max_depth=5,random_state=42),n_jobs=-1,random_state=42),
MLPClassifier(random_state=42,alpha=0.1),
         QuadraticDiscriminantAnalysis(reg_param=1)]

In [365]:
model_names = ['RandomForestClassifier',
               'AdaBoostClassifier',
               'BaggingClassifier',
               'MLPClassifier',
               'QuadraticDiscriminantAnalysis'
]

In [366]:
for model, model_name in zip(models, model_names):
    print('Training',model_name)
    t0 = time.time()
    model.fit(X_train,y_train)
    print('Training took',time.time()-t0)
    dump(model,'../glaucoma_project/Models/'+model_name+'_clinical_3_model_v2.joblib')

Training RandomForestClassifier
Training took 6.581325054168701
Training AdaBoostClassifier
Training took 95.44723296165466
Training BaggingClassifier
Training took 15.304724931716919
Training MLPClassifier
Training took 142.0470733642578
Training QuadraticDiscriminantAnalysis
Training took 8.96567988395691


In [367]:
train_stats = {}
val_stats = {}
for model, model_name in zip(models, model_names):
    print(model_name)
    if model_name != 'KNeighborsClassifier':
        print('Training:')
        try:
            y_train_pred_proba = model.predict_proba(X_train)
        except:
            y_train_pred_proba = model._predict_proba_lr(X_train)
        train_stats[model_name] = evaluate_model(y_train,y_train_pred_proba)
    print('Validation:')
    try:
        y_val_pred_proba = model.predict_proba(X_val)
    except:
        y_val_pred_proba = model._predict_proba_lr(X_val)
    val_stats[model_name] = evaluate_model(y_val,y_val_pred_proba)
    print('==========')

RandomForestClassifier
Training:
AUC 0.737373564295728
Youden's J threshold 0.5029385118896673
Accuracy 0.5902486752545284
Sensitivity 0.7709690553745928
Specificity 0.5880054885768363
Validation:
AUC 0.7119700205648296
Youden's J threshold 0.4938119090548381
Accuracy 0.5617081498786953
Sensitivity 0.7764505119453925
Specificity 0.5588436148417938
AdaBoostClassifier
Training:
AUC 0.7413111763059987
Youden's J threshold 0.49902714709788176
Accuracy 0.5698514637719865
Sensitivity 0.7899022801302932
Specificity 0.567120089151469
Validation:
AUC 0.7364330197980187
Youden's J threshold 0.49831365578865283
Accuracy 0.5192290412435978
Sensitivity 0.8276450511945392
Specificity 0.5151149556111997
BaggingClassifier
Training:
AUC 0.7370001662881467
Youden's J threshold 0.5106709038879227
Accuracy 0.5909400638473654
Sensitivity 0.7626221498371335
Specificity 0.5888090647434747
Validation:
AUC 0.7160483945526119
Youden's J threshold 0.46952999053578637
Accuracy 0.5409291041423309
Sensitivity 0.784

In [368]:
df_model_stats = pd.DataFrame()
df_model_stats['Model Names'] = model_names
df_model_stats['Train AUC'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][0] if x in train_stats.keys() else np.nan)
df_model_stats['Train Threshold'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][1] if x in train_stats.keys() else np.nan)
df_model_stats['Train Accuracy'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][2] if x in train_stats.keys() else np.nan)
df_model_stats['Train Sensitivity'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][3] if x in train_stats.keys() else np.nan)
df_model_stats['Train Specificity'] = df_model_stats['Model Names'].apply(lambda x: train_stats[x][4] if x in train_stats.keys() else np.nan)
df_model_stats['Val AUC'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][0] if x in val_stats.keys() else np.nan)
df_model_stats['Val Threshold'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][1] if x in val_stats.keys() else np.nan)
df_model_stats['Val Accuracy'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][2] if x in val_stats.keys() else np.nan)
df_model_stats['Val Sensitivity'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][3] if x in val_stats.keys() else np.nan)
df_model_stats['Val Specificity'] = df_model_stats['Model Names'].apply(lambda x: val_stats[x][4] if x in val_stats.keys() else np.nan)

In [369]:
df_model_stats.sort_values('Val AUC', ascending=False).to_csv('../glaucoma_project/UKBB_Data/processed_data/initial_model_selection_clinical_data_maximal_only_supplemental.csv', index=False)

Best model was GradientBoostingClassifier again...